In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
from ensemble_processing import load_data, load, save
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.dataframe_functions import roll_time_series

/Users/neilkloot/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/neilkloot/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import tsfresh
print(tsfresh.__version__)

0.11.2


In [3]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500

In [ ]:
extract_feature_set = {'640106_A3597525W': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5},
   {'r': 6},
   {'r': 7}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'FIRMMCRT': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5},
   {'r': 6},
   {'r': 7}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'FXRUSD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_crossing_m': [{'m': 1}],
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}, {'max': 1, 'min': -1}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None},
 'GRCPAIAD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'GRCPAISAD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5},
   {'r': 6},
   {'r': 7}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'GRCPBCAD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'GRCPBCSAD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5},
   {'r': 6},
   {'r': 7}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'GRCPBMAD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'GRCPNRAD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'GRCPRCAD': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'H01_GGDPCVGDPFY': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'angle'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_crossing_m': [{'m': 0}, {'m': 1}],
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}, {'max': 1, 'min': -1}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None},
 'H01_GGDPCVGDP': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'H05_GLFSEPTPOP': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None},
 'adjustedPrice': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_crossing_m': [{'m': 1}],
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}, {'max': 1, 'min': -1}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5},
   {'r': 6},
   {'r': 7}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'allordpreviousclose': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'asxpreviousclose': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
   {'num_segments': 10, 'segment_focus': 1},
   {'num_segments': 10, 'segment_focus': 2},
   {'num_segments': 10, 'segment_focus': 3},
   {'num_segments': 10, 'segment_focus': 4},
   {'num_segments': 10, 'segment_focus': 5},
   {'num_segments': 10, 'segment_focus': 6},
   {'num_segments': 10, 'segment_focus': 7},
   {'num_segments': 10, 'segment_focus': 8},
   {'num_segments': 10, 'segment_focus': 9}],
  'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'index_mass_quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'sumVolume': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_crossing_m': [{'m': 0}, {'m': 1}],
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}, {'max': 1, 'min': -1}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5},
   {'r': 6},
   {'r': 7}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'value_count': [{'value': 0}],
  'variance': None,
  'variance_larger_than_standard_deviation': None},
 'volume': {'abs_energy': None,
  'absolute_sum_of_changes': None,
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'approximate_entropy': [{'m': 2, 'r': 0.1},
   {'m': 2, 'r': 0.3},
   {'m': 2, 'r': 0.5},
   {'m': 2, 'r': 0.7},
   {'m': 2, 'r': 0.9}],
  'binned_entropy': [{'max_bins': 10}],
  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}],
  'cid_ce': [{'normalize': False}, {'normalize': True}],
  'count_above_mean': None,
  'count_below_mean': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
   {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'},
   {'coeff': 0, 'attr': 'real'}],
  'first_location_of_maximum': None,
  'first_location_of_minimum': None,
  'has_duplicate': None,
  'has_duplicate_max': None,
  'has_duplicate_min': None,
  'large_standard_deviation': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45}],
  'last_location_of_maximum': None,
  'last_location_of_minimum': None,
  'length': None,
  'linear_trend': [{'attr': 'rvalue'}],
  'longest_strike_above_mean': None,
  'longest_strike_below_mean': None,
  'maximum': None,
  'mean': None,
  'median': None,
  'minimum': None,
  'number_crossing_m': [{'m': 0}, {'m': 1}],
  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
  'number_peaks': [{'n': 1}, {'n': 10}, {'n': 3}, {'n': 5}],
  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
  'percentage_of_reoccurring_values_to_all_values': None,
  'quantile': [{'q': 0.1},
   {'q': 0.2},
   {'q': 0.3},
   {'q': 0.4},
   {'q': 0.6},
   {'q': 0.7},
   {'q': 0.8},
   {'q': 0.9}],
  'range_count': [{'max': 1000000000000.0, 'min': 0}, {'max': 1, 'min': -1}],
  'ratio_beyond_r_sigma': [{'r': 0.5},
   {'r': 1},
   {'r': 1.5},
   {'r': 2},
   {'r': 2.5},
   {'r': 3},
   {'r': 5},
   {'r': 6},
   {'r': 7}],
  'ratio_value_number_to_time_series_length': None,
  'standard_deviation': None,
  'sum_of_reoccurring_data_points': None,
  'sum_of_reoccurring_values': None,
  'sum_values': None,
  'symmetry_looking': [{'r': 0.05},
   {'r': 0.1},
   {'r': 0.15000000000000002},
   {'r': 0.2},
   {'r': 0.25},
   {'r': 0.30000000000000004},
   {'r': 0.35000000000000003},
   {'r': 0.4},
   {'r': 0.45},
   {'r': 0.5},
   {'r': 0.55},
   {'r': 0.6000000000000001},
   {'r': 0.65},
   {'r': 0.7000000000000001},
   {'r': 0.75},
   {'r': 0.8},
   {'r': 0.8500000000000001},
   {'r': 0.9},
   {'r': 0.9500000000000001}],
  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
  'value_count': [{'value': 0}],
  'variance': None,
  'variance_larger_than_standard_deviation': None}}

In [7]:
def fix_duplicate_columns(df):
    # Get unique list of columms
    unique_cols = np.unique(df.columns.values)
    
    for col in unique_cols:
        column_numbers = [x for x in range(df.shape[1])]  # list of columns' integer indices
        remove_index = -1
        already_located = False

        for col_num in range(len(df.columns)):
            if df.columns[col_num] == col and already_located:
                remove_index = col_num
                print('Found duplicate for ', col, '- remove index', remove_index)
            elif df.columns[col_num] == col and not already_located:
                already_located = True
#                 print('Found first occurence', col, '-', col_num)
                
        # If a duplicate has been found, remove the column from the index list
        if remove_index != -1:
            column_numbers.remove(remove_index) #removing column integer index n
            df = df.iloc[:, column_numbers] #return all columns except the nth column

    return df


In [8]:
from optimise_dataframe import optimise_df

def retrieve_weekly_symbol_dfs(path, run_str=None, load_list=None):
    """
        Retrieves the infividual dataframes saved for weekly result during pre-processing and returns them 
    """
    symbol_dfs = []

    if load_list:
        # Pre-defined files to look for
        for symbol in load_list:
            symbol_df = pd.read_pickle(path + 'ml-weekly-symbol-' + symbol + '-' + run_str + '.pkl.gz', compression='gzip')
#             symbol_df = optimise_df(symbol_df)

            symbol_dfs.append(symbol_df)
    else:
        # Look for files using the path
        print('Checking for files from', path)
        # Return files in path
        file_list = glob.glob(path + 'ml-weekly-symbol-*' + run_str + '.pkl.gz')

        # load each model set
        for file_name in file_list:
            # retrieve data frame and append to list
            symbol_df = pd.read_pickle(file_name, compression='gzip')
            symbol_df = optimise_df(symbol_df)

            symbol_dfs.append(symbol_df)

    print('Concatenating symbol dfs')
    # Create empty data frame
    output_df = pd.concat(symbol_dfs)
    
    # Handle accidental column duplicated
    output_df = fix_duplicate_columns(output_df)

    print('Optimising symbol dfs')
    output_df = optimise_df(output_df)

    return output_df

In [9]:
from tsfresh import extract_relevant_features

# experiment_symbols = ['WAX', 'IVV', 'WESN', 'IAA', 'XRO', 'MTB', 'MXR', 'RCP', 'ISX', 'IMI']

experiment_symbols = ['GSW','VGAD','CUA','TIX','FLT','PTN','OVN','IRU','FDM','HDF','SVW','BWX',
                      'FPH','AGL','JHC','HIN','TRS','HGO','SGP','AVN','DOW','NVL','AOG','BGL',
                      'HSN','CSS','SWM','RIC','CMI','MKE','BCI','IBG','SWJ','IXP','MYX','RNY',
                      'CZA','SRV','VMT','CDM','CLQ','BPS','CSE','ORR','OOK','EOS','BRL','BWP',
                      'ENC','FCT','SIO','SZG','ECG','SXA','MHC','PHK','ICT','CLH','NTM','TAS',
                      'REV','CII','FND','ZGM','GOE','SSI','QMN','EAL','DSB','PNX','KRS','RXH',
                      'LAU','CNW','GSZ','IBC','SAS','EAS','OCC','AIS','INK','AIK','EVM','MSV',
                      'XST','GMR','WWI','JYC','BSM','VRX','TKL','WFE','ZMI','SHK','DGO','BD1',
                      'ALT','SES','RNO','MXC','IRC','GTR','AYM','RLC','IVG','MDG','MPE','MOT',
                      'OAR','HGL','DSE','VII','KTL']

all_df = retrieve_weekly_symbol_dfs('../data/symbols/', '20190416', experiment_symbols)


Concatenating symbol dfs
Found duplicate for  asxpreviousclose_T11_20P - remove index 149
Found duplicate for  asxpreviousclose_T1P - remove index 149
Found duplicate for  asxpreviousclose_T2_5P - remove index 149
Found duplicate for  asxpreviousclose_T6_10P - remove index 149
Optimising symbol dfs
Field: lastTradePriceOnly
Field: adjustedPrice
Field: volume
Field: previousClose
Field: change
Field: changeInPercent
Field: 52WeekHigh
Field: 52WeekLow
Field: changeFrom52WeekHigh
Field: changeFrom52WeekLow
Field: percebtChangeFrom52WeekHigh
Field: percentChangeFrom52WeekLow
Field: allordpreviousclose
Field: allordchange
Field: allorddayshigh
Field: allorddayslow
Field: allordpercebtChangeFrom52WeekHigh
Field: allordpercentChangeFrom52WeekLow
Field: asxpreviousclose
Field: asxchange
Field: asxdayshigh
Field: asxdayslow
Field: asxpercebtChangeFrom52WeekHigh
Field: asxpercentChangeFrom52WeekLow
Field: exDividendRelative
Field: exDividendPayout
Field: 640106_A3597525W
Field: AINTCOV
Field: Be

In [10]:
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features, extract_relevant_features, select_features
from sklearn.impute import SimpleImputer

## Create dataframes for tsfresh
TSFRESH_VALUE_COLUMNS = ['adjustedPrice', 'volume', 'previousClose', 'change', 'changeInPercent', '52WeekHigh', 
                       '52WeekLow', 'changeFrom52WeekHigh', 'changeFrom52WeekLow', 'percebtChangeFrom52WeekHigh', 
                       'percentChangeFrom52WeekLow', 'allordpreviousclose', 'allordchange', 'allorddayshigh', 
                       'allorddayslow', 'allordpercebtChangeFrom52WeekHigh', 'allordpercentChangeFrom52WeekLow', 
                       'asxpreviousclose', 'asxchange', 'asxdayshigh', 'asxdayslow', 'asxpercebtChangeFrom52WeekHigh', 
                       'asxpercentChangeFrom52WeekLow', '640106_A3597525W', 'FIRMMCRT', 'FXRUSD', 'GRCPAIAD', 
                       'GRCPAISAD', 'GRCPBCAD', 'GRCPBCSAD', 'GRCPBMAD', 'GRCPNRAD', 'GRCPRCAD', 'H01_GGDPCVGDP', 
                       'H01_GGDPCVGDPFY', 'H05_GLFSEPTPOP', 'daysHigh', 'daysLow', 'sumVolume']

TSFRESH_COLUMN_ID = 'symbol'
TSFRESH_COLUMN_SORT = 'quoteDate'

ALL_TSFRESH_COLUMNS = TSFRESH_VALUE_COLUMNS.copy()
ALL_TSFRESH_COLUMNS.append(TSFRESH_COLUMN_ID)
ALL_TSFRESH_COLUMNS.append(TSFRESH_COLUMN_SORT)

# 8 week percentage return
working_df = all_df
target_df = pd.DataFrame()
working_df.drop('lastTradePriceOnly', axis=1, inplace=True, errors='ignore')

# Create targets
working_df['eight_week_return'] = (working_df['adjustedPrice'].shift(-8) - working_df['adjustedPrice'])  \
                                /  working_df['adjustedPrice'].clip(lower=0.1) * \
                                                100

# Drop rows missing target
working_df.dropna(subset=['eight_week_return'], inplace=True)
working_df['quoteDate'] = working_df.index


# Create separate target df
target_df[['symbol', 'quoteDate', 'eight_week_return']] = working_df[['symbol', 'quoteDate', 'eight_week_return']]


tsfresh_df = working_df[ALL_TSFRESH_COLUMNS]

ts_fresh_imputer = SimpleImputer(strategy='median')
tsfresh_df[TSFRESH_VALUE_COLUMNS] = ts_fresh_imputer.fit_transform(tsfresh_df[TSFRESH_VALUE_COLUMNS].values)



/Users/neilkloot/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/neilkloot/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
print('tsfresh', tsfresh_df.shape)
print('target', target_df.shape)


tsfresh (54843, 41)
target (54843, 3)


In [12]:
def make_forecasting_frame_nk(x, kind, max_timeshift, rolling_direction):
    """
    Takes a singular time series x and constructs a DataFrame df and target vector y that can be used for a time series
    forecasting task.
    The returned df will contain, for every time stamp in x, the last max_timeshift data points as a new
    time series, such can be used to fit a time series forecasting model.
    See :ref:`forecasting-label` for a detailed description of the rolling process and how the feature matrix and target
    vector are derived.
    The returned time series container df, will contain the rolled time series as a flat data frame, the first format
    from :ref:`data-formats-label`.
    When x is a pandas.Series, the index will be used as id.
    :param x: the singular time series
    :type x: np.array or pd.Series
    :param kind: the kind of the time series
    :type kind: str
    :param rolling_direction: The sign decides, if to roll backwards (if sign is positive) or forwards in "time"
    :type rolling_direction: int
    :param max_timeshift: If not None, shift only up to max_timeshift. If None, shift as often as possible.
    :type max_timeshift: int
    :return: time series container df, target vector y
    :rtype: (pd.DataFrame, pd.Series)
    """
    n = len(x)

    if isinstance(x, pd.Series):
        t = x.index
    else:
        t = range(n)

    df = pd.DataFrame({"id": ["id"] * n,
                       "time": t,
                       "value": x,
                       "kind": kind})

    df_shift = roll_time_series(df,
                                column_id="id",
                                column_sort="time",
                                column_kind="kind",
                                rolling_direction=rolling_direction,
                                max_timeshift=max_timeshift)

    return df_shift

In [13]:
TSFRESH_INITIAL_VALUE = 'adjustedPrice'
TSFRESH_COLUMNS = ['volume', 'allordpreviousclose', 'asxpreviousclose', '640106_A3597525W', 'FIRMMCRT', 
                   'FXRUSD', 'GRCPAIAD', 'GRCPAISAD', 'GRCPBCAD', 'GRCPBCSAD', 'GRCPBMAD', 'GRCPNRAD', 
                   'GRCPRCAD', 'H01_GGDPCVGDP', 'H01_GGDPCVGDPFY', 'H05_GLFSEPTPOP', 'sumVolume']

# Make base dataframe
df_shifted = make_forecasting_frame_nk(tsfresh_df[TSFRESH_INITIAL_VALUE], kind=tsfresh_df['symbol'], 
                                       max_timeshift=52, rolling_direction=1) 
df_shifted['unique_id'] = df_shifted['kind'].astype(str) + '_' + df_shifted['id'].dt.strftime('%Y-%m-%d')
df_shifted.rename(columns={'value': TSFRESH_INITIAL_VALUE}, inplace=True)

# Work through remaining cols
for col_name in TSFRESH_COLUMNS:
    working_shift = make_forecasting_frame_nk(tsfresh_df[col_name], kind=tsfresh_df['symbol'], 
                                              max_timeshift=52, rolling_direction=1) 
#     working_shift['unique_id'] = df_shift['kind'].astype(str) + '_' + df_shift['id'].dt.strftime('%Y-%m-%d')
    working_shift.rename(columns={'value': col_name}, inplace=True)

    df_shifted = df_shifted.merge(working_shift[['time', 'kind', 'id', col_name]], left_on=['time', 'kind', 'id'], 
                                  right_on=['time', 'kind', 'id'], how='left')


df_shifted.shape


/Users/neilkloot/anaconda/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:435: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "


(2750965, 22)

In [14]:
shift_target = target_df
shift_target['unique_id'] = shift_target['symbol'].astype(str) + '_' + shift_target['quoteDate'].dt.strftime('%Y-%m-%d')
shift_target.set_index('unique_id', inplace=True)                          
                          

In [15]:
print(df_shifted.columns)
print(df_shifted.shape)
print(target_df.columns)
print(shift_target.shape)

Index(['time', 'adjustedPrice', 'id', 'kind', 'unique_id', 'volume',
       'allordpreviousclose', 'asxpreviousclose', '640106_A3597525W',
       'FIRMMCRT', 'FXRUSD', 'GRCPAIAD', 'GRCPAISAD', 'GRCPBCAD', 'GRCPBCSAD',
       'GRCPBMAD', 'GRCPNRAD', 'GRCPRCAD', 'H01_GGDPCVGDP', 'H01_GGDPCVGDPFY',
       'H05_GLFSEPTPOP', 'sumVolume'],
      dtype='object')
(2750965, 22)
Index(['symbol', 'quoteDate', 'eight_week_return'], dtype='object')
(54843, 3)


In [ ]:
shift_target

In [ ]:
from tsfresh.feature_extraction import extract_features, ComprehensiveFCParameters

settings = ComprehensiveFCParameters()

# Create cols array
shifted_cols = ['time', 'adjustedPrice', 'unique_id']
shifted_cols.extend(TSFRESH_COLUMNS)
                

X = extract_features(df_shifted[shifted_cols], column_id='unique_id', column_sort='time', 
                     default_fc_parameters=settings, n_jobs=8)


Feature Extraction:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
print('Number of columns:', len(X.columns))
print(X.columns)
X.dropna(axis=1, inplace=True)
output_columns = X.columns
print('Removed NaN cols.  Remaining:', len(X.columns))
print(output_columns)

stats = pd.DataFrame()    
stats["Mean"]=X.mean()
stats["Std.Dev"]=X.std()
stats["Var"]=X.var()


In [ ]:
cols_to_remove = stats[(stats['Mean']==0) & (stats['Std.Dev']==0) & (stats['Var']==0)]

X.drop(cols_to_remove.index.values, axis=1, inplace=True)

X.shape

In [ ]:
from scipy import stats

col_stats = pd.DataFrame()

for col in X.columns:
    current_col_result = stats.kendalltau(X[col].values, shift_target['eight_week_return'].values)
    current_df = pd.DataFrame({"name": [col],
                               "correlation": [current_col_result[0]],
                               "p-value": [current_col_result[1]]})
    
    col_stats = col_stats.append(current_df)
    
col_stats


In [ ]:
col_stats.sort_values('p-value')

In [ ]:
# Look at feature importance using an xgb model
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split


importance_model = xgb.XGBRegressor(nthread=8, tree_method='auto', predictor='cpu_predictor', 
                                    n_estimators=250, max_depth=70, base_score=0.1, 
                                    colsample_bylevel=0.7, colsample_bytree=1.0, 
                                    gamma=0, learning_rate=0.05, min_child_weight=3)


print('Training xgboost importance model...')
x_train, x_test, y_train, y_test = train_test_split(X.values, shift_target['eight_week_return'].values, 
                                                    test_size=0.15)

eval_set = [(x_test, y_test)]
importance_model.fit(x_train, y_train, early_stopping_rounds=10, eval_metric='mae', eval_set=eval_set, verbose=True)


In [ ]:
feature_importance_df = pd.DataFrame({
    'column_name': X.columns,
    'importance': importance_model.feature_importances_
})

# feature_importance_df.sort_values('importance', ascending=False)
feature_importance_df[feature_importance_df['importance'] > 0].sort_values('importance', ascending=False)

In [ ]:
## USe sklearn RFE to detrmine best n features

from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

numberOfRequiredFeatures = 50
attributes = X
target = shift_target['eight_week_return'].values

def return_selected_features(attributes, selected_msk):
    selected_attributes = attributes.copy()
    index = 0
    indexNames = list(attributes.columns.values)
    for name in indexNames:
        if not selected_msk[index]:
            selected_attributes = selected_attributes.drop(name, axis=1)
        index = index + 1

    return selected_attributes.copy()

estimator = SVR(kernel="linear")
selector = RFE(estimator, numberOfRequiredFeatures, step=0.05)
selector = selector.fit(attributes, target)
svr_support = selector.support_
svr_ranking = selector.ranking_
svr_features = return_selected_features(attributes, selector.support_)
print('SVR')
print(svr_features)

estimator = BayesianRidge()
selector = RFE(estimator, numberOfRequiredFeatures, step=0.05)
selector = selector.fit(attributes, target)
br_support = selector.support_
br_ranking = selector.ranking_
br_features = return_selected_features(attributes, selector.support_)
print('Bayesian Ridge')
print(br_features)

estimator = LinearRegression()
selector = RFE(estimator, numberOfRequiredFeatures, step=0.05)
selector = selector.fit(attributes, target)
lr_support = selector.support_
lr_ranking = selector.ranking_
lr_features = return_selected_features(attributes, selector.support_)
print('Linear Regression')
print(lr_features)

kernel = DotProduct() + WhiteKernel()
estimator = GaussianProcessRegressor(kernel=kernel, random_state=0)
selector = RFE(estimator, numberOfRequiredFeatures, step=0.05)
selector = selector.fit(attributes, target)
gpr_support = selector.support_
gpr_ranking = selector.ranking_
gpr_features = return_selected_features(attributes, selector.support_)
print('Gaussian Process Regressor')
print(gpr_features)



In [ ]:
from tsfresh.feature_extraction import settings

settings.from_columns(X.columns)

In [ ]:
# X.describe()
pca_train = PCAForPandas(n_components=30)
X_train_pca = pca_train.fit_transform(X)

In [ ]:
X_train_pca

In [ ]:
# Look at feature importance using an xgb model
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split


pca_importance_model = xgb.XGBRegressor(nthread=8, tree_method='auto', predictor='cpu_predictor', 
                                        n_estimators=250, max_depth=70, base_score=0.1, 
                                        colsample_bylevel=0.7, colsample_bytree=1.0, 
                                        gamma=0, learning_rate=0.05, min_child_weight=3)


print('Training xgboost pca importance model...')
x_train, x_test, y_train, y_test = train_test_split(X_train_pca.values, shift_target['eight_week_return'].values, 
                                                    test_size=0.15)

eval_set = [(x_test, y_test)]
pca_importance_model.fit(x_train, y_train, early_stopping_rounds=10, eval_metric='mae', eval_set=eval_set, verbose=True)


In [ ]:
pca_feature_importance_df = pd.DataFrame({
    'column_name': X_train_pca.columns,
    'importance': pca_importance_model.feature_importances_
})

pca_feature_importance_df.sort_values('importance', ascending=False)
# feature_importance_df[feature_importance_df['importance'] > 0]

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd


class PCAForPandas(PCA):
    """This class is just a small wrapper around the PCA estimator of sklearn including normalization to make it 
    compatible with pandas DataFrames.
    """

    def __init__(self, **kwargs):
        self._z_scaler = StandardScaler()
        super(self.__class__, self).__init__(**kwargs)

        self._X_columns = None

    def fit(self, X, y=None):
        """Normalize X and call the fit method of the base class with numpy arrays instead of pandas data frames."""

        X = self._prepare(X)

        self._z_scaler.fit(X.values, y)
        z_data = self._z_scaler.transform(X.values, y)

        return super(self.__class__, self).fit(z_data, y)

    def fit_transform(self, X, y=None):
        """Call the fit and the transform method of this class."""

        X = self._prepare(X)

        self.fit(X, y)
        return self.transform(X, y)

    def transform(self, X, y=None):
        """Normalize X and call the transform method of the base class with numpy arrays instead of pandas data frames."""

        X = self._prepare(X)

        z_data = self._z_scaler.transform(X.values, y)

        transformed_ndarray = super(self.__class__, self).transform(z_data)

        pandas_df = pd.DataFrame(transformed_ndarray)
        pandas_df.columns = ["pca_{}".format(i) for i in range(len(pandas_df.columns))]

        return pandas_df

    def _prepare(self, X):
        """Check if the data is a pandas DataFrame and sorts the column names.

        :raise AttributeError: if pandas is not a DataFrame or the columns of the new X is not compatible with the 
                               columns from the previous X data
        """
        if not isinstance(X, pd.DataFrame):
            raise AttributeError("X is not a pandas DataFrame")

        X.sort_index(axis=1, inplace=True)

        if self._X_columns is not None:
            if self._X_columns != list(X.columns):
                raise AttributeError("The columns of the new X is not compatible with the columns from the previous X data")
        else:
            self._X_columns = list(X.columns)

        return X

In [ ]:
from tsfresh import select_features

X.dropna(axis=1, inplace=True)
X_filtered = select_features(X, shift_target['eight_week_return'])

In [ ]:
X.shape

In [ ]:
cols = np.unique(all_df.columns.values)

for col in cols:
    if 'future' in col.lower():
        print(col)

In [ ]:
from tsfresh import extract_features

def extract_tsfresh_features(df):
    symbol = df['symbol'].unique()
    print('Extracting features for', symbol)
    df_time_series = pd.DataFrame()
    df_time_series['symbol'] = df['symbol']
    df_time_series['quoteDate'] = df['quoteDate']
    df_time_series['adjustedPrice'] = df['adjustedPrice']
    df_time_series.dropna(axis=0, how='any', inplace=True)
    df_shift, y = make_forecasting_frame(df_time_series["adjustedPrice"], kind="return", 
                                         max_timeshift=8, rolling_direction=-1) 
#     print(df_shift)
    X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value",
                        default_fc_parameters=extract_feature_set)
#     X = extract_relevant_features(df_shift, y, column_id="id", column_sort="time", column_value="value", 
#                                   ml_task='regression', show_warnings=False)
    print(len(X.columns), 'tsfresh features created for', symbol)
    return X
    

def add_tsfresh_features(df):
    X_tsf = extract_tsfresh_features(df)
    X_tsf.fillna(0, inplace=True)
    symbol = df['symbol'].unique()
    print('Merging data')
    
    output_df = df.merge(X_tsf, left_on='quoteDate', right_on='id', how='left')
    
#     return X_tsf, output_df
    return output_df


In [ ]:
# Load and augment data
# all_df = pd.read_pickle('../data/ml-20180922-labelled.pkl.gz', compression='gzip')
# experiment_symbols = ['WAX', 'IVV', 'WESN', 'IAA', 'XRO', 'MTB', 'MXR', 'RCP', 'ISX', 'IMI']
experiment_symbols = ['GSW','CUA','FLT','OVN','FDM','SVW','FPH','JHC','TRS','SGP','DOW','AOG',
                      'HSN','SWM','CMI','BCI','SWJ','MYX','CZA','VMT','CLQ','CSE','OOK','BRL',
                      'ENC','SIO','ECG','MHC','ICT','NTM','REV','FND','GOE','QMN','DSB','KRS',
                      'LAU','GSZ','SAS','OCC','INK','EVM','XST','WWI','BSM','TKL','ZMI','DGO',
                      'ALT','RNO','IRC','AYM','IVG','MPE','OAR','DSE','KTL']


train_x_dfs = []
augmented_x_dfs = []
round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']


for symbol in experiment_symbols:
    # Filter to model data for this symbol and re-set the pandas indexes
    model_data = all_df.loc[all_df['symbol'] == symbol]
    
    print('Symbol:', symbol, 'number of records:', len(model_data))
        
    ts_model_data = add_tsfresh_features(model_data)
        
    # Create round masks to split into train / test sets
    for round_name in round_cols:
        # Create random mask with 75% values as training data, 12.5% as training validation,12.5% as holdout
        test_portion = 0.125
        
        # Get index values and length
#         print(model_data)
        df_ix = model_data.index.values
#         print(df_ix)
        len_ix = len(model_data)
        
        # Calculate position in indices for each portion
        holdout_num = len_ix - int(test_portion * len_ix)
        test_num = len_ix - (2*int(test_portion * len_ix))
        
        # Shuffle index values
        np.random.shuffle(df_ix)

        # Create array with randomised indices for train, test, holdout
        test_index = df_ix[test_num:holdout_num]
        holdout_index = df_ix[holdout_num:]

        model_data[round_name] = 'train'
        model_data.loc[test_index, round_name] = 'test'
        model_data.loc[holdout_index, round_name] = 'holdout'        
    
        # Run process again for ts data
        
        df_ix = ts_model_data.index.values
        len_ix = len(ts_model_data)
#         print('Total',len_ix)
        
        # Calculate position in indices for each portion
        holdout_num = len_ix - int(test_portion * len_ix)
        test_num = len_ix - (2*int(test_portion * len_ix))
#         print('Holdout num',holdout_num)
#         print('Holdout num',test_num)
        
        # Shuffle index values
        np.random.shuffle(df_ix)

        # Create array with randomised indices for train, test, holdout
        test_index = df_ix[test_num:holdout_num]
        holdout_index = df_ix[holdout_num:]
        
#         print('test_index')
#         print(test_index)
#         print('holdout_index')
#         print(holdout_index)

        ts_model_data[round_name] = 'train'
        ts_model_data.loc[test_index, round_name] = 'test'
        ts_model_data.loc[holdout_index, round_name] = 'holdout'   
    
        print(' - ', len(model_data.loc[model_data[round_name]=='train']), 'training records')
        print(' - ', len(model_data.loc[model_data[round_name]=='test']), 'training validation records')
        print(' - ', len(model_data.loc[model_data[round_name]=='holdout']), 'holdout records')
        
    train_x_dfs.append(model_data)
    augmented_x_dfs.append(ts_model_data)

# Create concatenated dataframe with all data
print('Creating concatenated dataframes')
df_train_x = pd.concat(train_x_dfs)
df_augmented_train_x = pd.concat(augmented_x_dfs)

print('-'*20)
print('Overall totals')
print(' - ', len(df_train_x.loc[df_train_x[round_name]=='train']), 'training records')
print(' - ', len(df_train_x.loc[df_train_x[round_name]=='test']), 'training validation records')
print(' - ', len(df_train_x.loc[df_train_x[round_name]=='holdout']), 'holdout records')


del train_x_dfs
del augmented_x_dfs
del all_df

In [ ]:
# Save dataframes
df_train_x.to_pickle("../data/ml-experiment-baseline.pkl.gz", compression='gzip')
df_augmented_train_x.to_pickle("../data/ml-experiment-tsfresh.pkl.gz", compression='gzip')

In [ ]:
print(df_train_x.columns.values)
# print(df_train_x.drop(['future_eight_week_return'], axis=1, errors='ignore').columns.values)

# Pre-processing

In [ ]:
from ensemble_processing import execute_preprocessor
from processing_constants import LABEL_COLUMN
from processing_constants import CONTINUOUS_COLUMNS, PAST_RESULTS_CONTINUOUS_COLUMNS
from processing_constants import CATEGORICAL_COLUMNS, PAST_RESULTS_CATEGORICAL_COLUMNS
from processing_constants import COLUMNS_TO_REMOVE, RECURRENT_COLUMNS


return_col = 'future_eight_week_return'
round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']

df_train_x.dropna(subset=[return_col], inplace=True)

all_y = df_train_x[return_col].values
all_train_x = df_train_x.drop([return_col], axis=1, errors='ignore')

all_train_x['exDividendRelative'] = all_train_x['exDividendDate'] - all_train_x['quoteDate']

# convert string difference value to integer
all_train_x['exDividendRelative'] = all_train_x['exDividendRelative'].apply(
    lambda x: np.nan if pd.isnull(x) else x.days)
# Make sure it is the minimum data type size
all_train_x.loc[:, 'exDividendRelative'] = all_train_x['exDividendRelative'].astype('int32', errors='ignore')

all_train_x['quoteDate_TIMESTAMP'] = (pd.DatetimeIndex(all_train_x['quoteDate']) - 
                                      pd.datetime(2007, 1, 1)).total_seconds()
all_train_x['quoteDate_TIMESTAMP'] = all_train_x['quoteDate_TIMESTAMP'].astype('int32', errors='ignore')
all_train_x['quoteDate_YEAR'] = pd.DatetimeIndex(all_train_x['quoteDate']).year.astype('str')
all_train_x['quoteDate_YEAR'] = all_train_x['quoteDate_YEAR'].astype('int32', errors='ignore')
all_train_x['quoteDate_MONTH'] = pd.DatetimeIndex(all_train_x['quoteDate']).month.astype('str')
all_train_x['quoteDate_MONTH'] = all_train_x['quoteDate_MONTH'].astype('int32', errors='ignore')
all_train_x['quoteDate_DAY'] = pd.DatetimeIndex(all_train_x['quoteDate']).day.astype('str')
all_train_x['quoteDate_DAY'] = all_train_x['quoteDate_DAY'].astype('int32', errors='ignore')
all_train_x['quoteDate_DAYOFWEEK'] = pd.DatetimeIndex(all_train_x['quoteDate']).dayofweek.astype('str')
all_train_x['quoteDate_DAYOFWEEK'] = all_train_x['quoteDate_DAYOFWEEK'].astype('int32', errors='ignore')

all_train_x.drop(['quoteDate', 'exDividendDate'], axis=1, inplace=True)

symbol_encoder = load('../models/se.pkl.gz')
imputer = load('../models/imputer.pkl.gz')
scaler = load('../models/scaler.pkl.gz')

print('Executing pre-processing')
# Execute pre-processing
df_pp_train_x = execute_preprocessor(all_train_x, symbol_encoder, imputer, scaler)


In [ ]:
from ensemble_processing import execute_preprocessor
from processing_constants import LABEL_COLUMN
from processing_constants import CONTINUOUS_COLUMNS, PAST_RESULTS_CONTINUOUS_COLUMNS
from processing_constants import CATEGORICAL_COLUMNS, PAST_RESULTS_CATEGORICAL_COLUMNS
from processing_constants import COLUMNS_TO_REMOVE, RECURRENT_COLUMNS


return_col = 'future_eight_week_return'
round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']

df_augmented_train_x.dropna(subset=[return_col], inplace=True)
aug_train_x = df_augmented_train_x

aug_train_x['exDividendRelative'] = aug_train_x['exDividendDate'] - aug_train_x['quoteDate']

# convert string difference value to integer
aug_train_x['exDividendRelative'] = aug_train_x['exDividendRelative'].apply(
    lambda x: np.nan if pd.isnull(x) else x.days)
# Make sure it is the minimum data type size
aug_train_x.loc[:, 'exDividendRelative'] = aug_train_x['exDividendRelative'].astype('int32', errors='ignore')

aug_train_x['quoteDate_TIMESTAMP'] = (pd.DatetimeIndex(aug_train_x['quoteDate']) - 
                                      pd.datetime(2007, 1, 1)).total_seconds()
aug_train_x['quoteDate_TIMESTAMP'] = aug_train_x['quoteDate_TIMESTAMP'].astype('int32', errors='ignore')
aug_train_x['quoteDate_YEAR'] = pd.DatetimeIndex(aug_train_x['quoteDate']).year.astype('str')
aug_train_x['quoteDate_YEAR'] = aug_train_x['quoteDate_YEAR'].astype('int32', errors='ignore')
aug_train_x['quoteDate_MONTH'] = pd.DatetimeIndex(aug_train_x['quoteDate']).month.astype('str')
aug_train_x['quoteDate_MONTH'] = aug_train_x['quoteDate_MONTH'].astype('int32', errors='ignore')
aug_train_x['quoteDate_DAY'] = pd.DatetimeIndex(aug_train_x['quoteDate']).day.astype('str')
aug_train_x['quoteDate_DAY'] = aug_train_x['quoteDate_DAY'].astype('int32', errors='ignore')
aug_train_x['quoteDate_DAYOFWEEK'] = pd.DatetimeIndex(aug_train_x['quoteDate']).dayofweek.astype('str')
aug_train_x['quoteDate_DAYOFWEEK'] = aug_train_x['quoteDate_DAYOFWEEK'].astype('int32', errors='ignore')

aug_train_x.drop(['quoteDate', 'exDividendDate'], axis=1, inplace=True)


aug_train_x.drop(['GICSSector', 'GICSIndustryGroup', 'GICSIndustry', 'one_week_comparison_date',
                  'two_week_comparison_date', 'four_week_comparison_date',
                  'eight_week_comparison_date', 'twelve_week_comparison_date',
                  'twenty_six_week_comparison_date', 'fifty_two_week_comparison_date'], 
                  axis=1, inplace=True)

symbol_encoder = load('../models/se.pkl.gz')
imputer = load('../models/imputer.pkl.gz')
scaler = load('../models/scaler.pkl.gz')

print('Executing pre-processing')
# Execute pre-processing
df_pp_augmented_train_x = execute_preprocessor(aug_train_x, symbol_encoder, imputer, scaler)

aug_y = df_pp_augmented_train_x[return_col].values
df_pp_augmented_train_x.drop([return_col], axis=1, errors='ignore', inplace=True)

## Baseline

In [ ]:
df_train_x.isnull().sum()

In [ ]:
df_pp_train_x.drop(['GICSSector', 'GICSIndustryGroup', 'GICSIndustry', 'one_week_comparison_date',
                   'two_week_comparison_date', 'four_week_comparison_date',
                   'eight_week_comparison_date', 'twelve_week_comparison_date',
                   'twenty_six_week_comparison_date', 'fifty_two_week_comparison_date'], axis=1, inplace=True)

df_pp_train_x.dtypes


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

def execute_baseline_model():
    baseline_model = xgb.XGBRegressor(nthread=8, tree_method='auto', predictor='cpu_predictor', 
                                      n_estimators=250, max_depth=70, base_score=0.1, 
                                      colsample_bylevel=0.7, colsample_bytree=1.0, 
                                      gamma=0, learning_rate=0.05, min_child_weight=3)


    print('Training xgboost baseline model...')
    x_train, x_test, y_train, y_test = train_test_split(df_pp_train_x.values, train_y_values, test_size=0.15)

    eval_set = [(x_test, y_test)]
    baseline_model.fit(x_train, y_train, early_stopping_rounds=25, eval_metric='mae', eval_set=eval_set, verbose=True)
    return baseline_model

def execute_baseline_kfold_model(x_train, y_train,  x_test, y_test):
    baseline_model = xgb.XGBRegressor(nthread=8, tree_method='auto', predictor='cpu_predictor', 
                                      n_estimators=150, max_depth=70, base_score=0.1, 
                                      colsample_bylevel=0.7, colsample_bytree=1.0, 
                                      gamma=0, learning_rate=0.05, min_child_weight=3)


    print('Training xgboost baseline kfold model...')
    eval_set = [(x_test, y_test)]
    baseline_model.fit(x_train, y_train, early_stopping_rounds=25, eval_metric='mae', eval_set=eval_set, verbose=True)
    return baseline_model

In [ ]:
# print(len(all_train_x))
print(len(df_pp_train_x))

df_pp_train_x.isnull().sum()


# msk = np.random.rand(len(df_pp_train_x)) < 0.8
# len(msk[msk==False])

In [ ]:
round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']
round_train = 'round_0'
train_mask = df_pp_train_x[round_name]
# train_x = df_pp_train_x[train_mask] 
# train_x
# test_x = df_pp_train_x[~train_mask].drop(round_cols, axis=1)
# test_x
train_y = all_y[train_mask]
len(train_y)
# test_y = all_y[~train_mask]
# len(test_y)

# msk = np.random.rand(len(df_pp_train_x)) < 0.8
# df_pp_train_x['round_1'] = msk

df_pp_train_x[round_cols]

In [ ]:
results = []

round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']

for round_name in round_cols:
    # Get x, y train / test for this round    
    train_mask = df_pp_train_x[round_name]
    train_x = df_pp_train_x[train_mask].drop(round_cols, axis=1).values
    train_y = all_y[train_mask]
    test_x = df_pp_train_x[~train_mask].drop(round_cols, axis=1).values
    test_y = all_y[~train_mask]
    
    # Run model with data removing the round test / train values
    model = execute_baseline_kfold_model(train_x, train_y, test_x, test_y) 

    execution_results = model.evals_result()
    best_result = np.min(execution_results['validation_0']['mae'])
    print(round_name, 'result:', best_result)        
    results.append(best_result)
    
print('Average result:',np.mean(results))

In [ ]:
# from sklearn.model_selection import KFold

# results = []

# kf = KFold(n_splits=5, shuffle=True, random_state=3072009)

# for k, (train, test) in enumerate(kf.split(df_pp_train_x.values, train_y_values)):
#     model = execute_baseline_kfold_model(df_pp_train_x.drop(round_cols, axis=1).values[train], 
#                                          train_y_values[train], 
#                                          df_pp_train_x.drop(round_cols, axis=1).values[test], 
#                                          train_y_values[test]) 

#     execution_results = model.evals_result()
#     best_result = np.min(execution_results['validation_0']['mae'])
#     print('Round', k, 'result:', best_result)        
#     results.append(best_result)

# print('Average result:',np.mean(results))

# n_estimators = 250, Average result: 11.1293088

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_importance

baseline_columns = df_pp_train_x.drop(round_cols, axis=1).columns.values
plt.figure(num=None, figsize=(100, 10), dpi=80, facecolor='w', edgecolor='k')
plt.bar(range(len(model.feature_importances_)), model.feature_importances_)
plt.xticks(range(len(model.feature_importances_)), baseline_columns, rotation='vertical');

In [ ]:
print(len(baseline_columns))
print(len(model.feature_importances_))

In [ ]:
base_feature_importance_df = pd.DataFrame({
    'column_name': baseline_columns,
    'importance': model.feature_importances_
})

In [ ]:
base_feature_importance_df.sort_values(by=['importance'], ascending=False, inplace=True)
base_feature_importance_df

In [ ]:
import matplotlib.pyplot as plt2
%matplotlib inline

plt2.figure(num=None, figsize=(10, 100), dpi=80, facecolor='w', edgecolor='k')
plt2.barh(range(len(base_feature_importance_df['importance'].values)), 
          base_feature_importance_df['importance'].values)
plt2.yticks(range(len(base_feature_importance_df['column_name'].values)), 
            base_feature_importance_df['column_name'].values, 
            rotation='horizontal')
plt2.gca().invert_yaxis()

## TSFresh

In [ ]:
ALL_PREVIOUS_COLUMNS = []
ALL_PREVIOUS_COLUMNS.extend(CONTINUOUS_COLUMNS)
ALL_PREVIOUS_COLUMNS.extend(PAST_RESULTS_CONTINUOUS_COLUMNS)
ALL_PREVIOUS_COLUMNS.extend(RECURRENT_COLUMNS)
# ALL_PREVIOUS_COLUMNS.extend(ALL_TSFRESH)

# # print(ALL_PREVIOUS_COLUMNS)

mask = np.isin(df_augmented_train_x.columns.values, ALL_PREVIOUS_COLUMNS)
# print(mask)

OTHER_COLS = df_augmented_train_x.columns[~mask].values
OTHER_COLS

# df_augmented_train_x.isnull().sum() #.any()
# df_augmented_train_x[ALL_TSFRESH].isnull().sum() #.any()
df_augmented_train_x[TSFRESH_RELEVANT].isnull().sum() #.any()
# df_augmented_train_x[OTHER_COLS].isnull().sum() #.any()
# df_augmented_train_x[CONTINUOUS_COLUMNS].isnull().sum() #.any()
# df_augmented_train_x[PAST_RESULTS_CONTINUOUS_COLUMNS].isnull().sum() #.any()
# df_augmented_train_x[RECURRENT_COLUMNS].isnull().sum() #.any()


In [ ]:
df_augmented_train_x.columns.values

In [ ]:
# msk = np.random.rand(len(df_augmented_train_x)) < 0.8
# df_augmented_train_x['round_1'] = msk
# df_augmented_train_x.drop(['round_!'], axis=1, inplace=True)

In [ ]:
# df_augmented_train_x[ALL_TSFRESH] = df_augmented_train_x[ALL_TSFRESH].fillna(0)

for val in TSFRESH_RELEVANT:
    if val in df_augmented_train_x.columns.values:
        print(val,'found')
        df_augmented_train_x[val] = df_augmented_train_x[val].fillna(0)
    else:
        print(val,'not found')

In [ ]:
df_pp_augmented_train_x.describe()

In [ ]:
len(aug_y)

In [ ]:
df_pp_train_x[round_cols]

In [ ]:
aug_y

In [ ]:
df_pp_augmented_train_x.describe()

In [ ]:
df_pp_augmented_train_x.isnull().sum()

In [ ]:
df_pp_augmented_train_x.dtypes


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

def execute_augmented_model():
    augmented_model = xgb.XGBRegressor(nthread=8, tree_method='auto', predictor='cpu_predictor', 
                                       n_estimators=250, max_depth=70, base_score=0.1, 
                                       colsample_bylevel=0.7, colsample_bytree=1.0, 
                                       gamma=0, learning_rate=0.05, min_child_weight=3)


    print('Training xgboost augmented model...')
    x_train, x_test, y_train, y_test = train_test_split(df_pp_augmented_train_x.values, 
                                                        train_y_augmented_values, test_size=0.15)

    eval_set = [(x_test, y_test)]
    augmented_model.fit(x_train, y_train, early_stopping_rounds=25, eval_metric='mae', eval_set=eval_set, verbose=True)
    return augmented_model

def execute_augmented_kfold_model(x_train, y_train,  x_test, y_test):
    augmented_model = xgb.XGBRegressor(nthread=8, tree_method='auto', predictor='cpu_predictor', 
                                      n_estimators=150, max_depth=90, base_score=0.1, 
                                      colsample_bylevel=0.7, colsample_bytree=1.0, 
                                      gamma=0, learning_rate=0.075, min_child_weight=3)


    print('Training xgboost augmented kfold model...')
    eval_set = [(x_test, y_test)]
    augmented_model.fit(x_train, y_train, early_stopping_rounds=25, eval_metric='mae', eval_set=eval_set, verbose=True)
    return augmented_model

In [ ]:
results = []

round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']

for round_name in round_cols:
    # Get x, y train / test for this round    
    train_mask = df_pp_augmented_train_x[round_name]
    train_x = df_pp_augmented_train_x[train_mask].drop(round_cols, axis=1).values
    train_y = aug_y[train_mask]
    test_x = df_pp_augmented_train_x[~train_mask].drop(round_cols, axis=1).values
    test_y = aug_y[~train_mask]
    
    # Run model with data removing the round test / train values
    augmented_model = execute_augmented_kfold_model(train_x, train_y, test_x, test_y) 

    execution_results = augmented_model.evals_result()
    best_result = np.min(execution_results['validation_0']['mae'])
    print(round_name, 'result:', best_result)        
    results.append(best_result)
    
print('Average result:',np.mean(results))

In [ ]:
# from sklearn.model_selection import KFold

# results = []

# kf = KFold(n_splits=5, shuffle=True, random_state=3072009)

# for k, (train, test) in enumerate(kf.split(df_pp_augmented_train_x.values, train_y_augmented_values)):
#     model = execute_augmented_kfold_model(df_pp_augmented_train_x.values[train], 
#                                          train_y_augmented_values[train], 
#                                          df_pp_augmented_train_x.values[test], 
#                                          train_y_augmented_values[test]) 

#     execution_results = model.evals_result()
#     best_result = np.min(execution_results['validation_0']['mae'])
#     print('Round', k, 'result:', best_result)        
#     results.append(best_result)

# print('Average result:',np.mean(results))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_importance

augmented_columns = df_pp_augmented_train_x.drop(round_cols, axis=1).columns.values
plt.figure(num=None, figsize=(100, 10), dpi=80, facecolor='w', edgecolor='k')
plt.bar(range(len(augmented_model.feature_importances_)), augmented_model.feature_importances_)
plt.xticks(range(len(augmented_model.feature_importances_)), augmented_columns, rotation='vertical');

In [ ]:
augmented_columns

In [ ]:
feature_importance_df = pd.DataFrame({
    'column_name': augmented_columns,
    'importance': augmented_model.feature_importances_
})

In [ ]:
feature_importance_df.sort_values(by=['importance'], ascending=False, inplace=True)
# feature_importance_df.to_csv('./feature-importances.csv')
feature_importance_df

In [ ]:
import matplotlib.pyplot as plt2
%matplotlib inline

plt2.figure(num=None, figsize=(10, 100), dpi=80, facecolor='w', edgecolor='k')
plt2.barh(range(len(feature_importance_df['importance'].values)), feature_importance_df['importance'].values)
plt2.yticks(range(len(feature_importance_df['column_name'].values)), feature_importance_df['column_name'].values, 
           rotation='horizontal')
plt2.gca().invert_yaxis()

## Deep learning

In [ ]:
# Scale tsfresh cols
from sklearn.preprocessing import MinMaxScaler, Imputer

tsfresh_cols = []
scaled_df_pp_augmented_train_x = df_pp_augmented_train_x

print('Starting stats')
print('  - num of NaN vals:', scaled_df_pp_augmented_train_x.isnull().sum())
# print('  - num of infinite vals:', np.isinf(scaled_df_pp_augmented_train_x.values).sum())
print('-'*20)


# Create array of columns and make sure infinite values are replaced by NaN
for col in df_pp_augmented_train_x.columns:
    if '__' in col:
        print(col)
        tsfresh_cols.append(col)
        print('  - num of NaN vals:', scaled_df_pp_augmented_train_x[col].isnull().sum())
        print('  - num of infinite vals:', np.isinf(scaled_df_pp_augmented_train_x[col].values).sum())
        scaled_df_pp_augmented_train_x[col] = scaled_df_pp_augmented_train_x[col].mask(np.isinf(scaled_df_pp_augmented_train_x[col]))

print('Num of tsfresh cols:', len(tsfresh_cols))

print(scaled_df_pp_augmented_train_x[tsfresh_cols].isnull().sum())
print(np.isinf(scaled_df_pp_augmented_train_x[tsfresh_cols]).sum())

ts_fresh_imputer = Imputer(strategy='median')
ts_fresh_imputer.fit(scaled_df_pp_augmented_train_x[tsfresh_cols].values)
scaled_df_pp_augmented_train_x[tsfresh_cols] = ts_fresh_imputer.transform(scaled_df_pp_augmented_train_x[tsfresh_cols].values)
scaled_df_pp_augmented_train_x[tsfresh_cols] = ts_fresh_imputer.transform(scaled_df_pp_augmented_train_x[tsfresh_cols].values)

print(scaled_df_pp_augmented_train_x[tsfresh_cols].isnull().sum())

tsfresh_scaler = MinMaxScaler(feature_range=(0, 1))
tsfresh_scaler.fit(scaled_df_pp_augmented_train_x[tsfresh_cols].values)
scaled_df_pp_augmented_train_x[tsfresh_cols] = tsfresh_scaler.transform(scaled_df_pp_augmented_train_x[tsfresh_cols].values)
scaled_df_pp_augmented_train_x[tsfresh_cols] = tsfresh_scaler.transform(scaled_df_pp_augmented_train_x[tsfresh_cols].values)

print('-'*20)
print('Final stats')
print('  - num of NaN vals:',scaled_df_pp_augmented_train_x.isnull().sum())
# print('  - num of infinite vals:',np.isinf(scaled_df_pp_augmented_train_x).sum())



In [ ]:
np.isinf(scaled_df_pp_augmented_train_x[col]).sum()

In [ ]:
round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']
dtypes = ['float32', 'float64', 'int8']

def remove_cols(df):
    for col in df.columns:
        if col not in round_cols and df[col].dtype in dtypes:
            print('Keeping col: ', col, df[col].dtype)  
        else:
            print('Removing col: ', col, df[col].dtype) 
            df.drop([col], axis=1, inplace=True)
            
    return df

In [ ]:
from keras import Model
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from compile_keras import compile_keras_model
from clr_callback import CyclicLR
from eval_results import eval_results 
from stats_operations import safe_log, safe_exp
from scipy import stats


def train_nn_k_fold(x_train, y_train, x_test, y_test):

    network = {
            'hidden_layers': [7, 7, 7, 7],
            'activation': 'relu',
            'optimizer': 'Adamax',
            'kernel_initializer': 'normal',
            'dropout': 0.1,
            'batch_size': 512,
            'model_type': 'mae',
            'int_layer': 30,
        }

    dimensions = x_train.shape[1]

    model = compile_keras_model(network, dimensions)

    early_stopping = EarlyStopping(monitor='val_loss', patience=30)
    csv_logger = CSVLogger('./logs/nn-training-log.log')
    checkpointer = ModelCheckpoint(
        filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)

    print('Training keras mae model...')

    # Reorder array - get array index
    array_index = np.arange(x_train.shape[0])

    step_size = math.ceil(array_index.shape[0] * .85 / 512) * 100
    clr = CyclicLR(base_lr=0.001, max_lr=0.04,
                   step_size=step_size, mode='exp_range', gamma=0.96)

    # Reshuffle index
    np.random.shuffle(array_index)

    # Create array using new index
    x_shuffled_train = x_train[array_index]
    y_shuffled_train = safe_log(y_train[array_index])
    

    # history = model.fit(x_shuffled_train,
    model.fit(x_shuffled_train,
              y_shuffled_train,
              validation_split=0.15,
              epochs=1000,
              batch_size=network['batch_size'],
              callbacks=[  # reduce_lr,
                  early_stopping, checkpointer, csv_logger, clr],
              verbose=0)

    model.load_weights('./weights/nn-weights.hdf5')

    print('Executing keras predictions...')

    log_y_predictions = model.predict(x_test)
    exp_predictions = safe_exp(log_y_predictions)

    pred_result = eval_results({'keras_log_y': {
            'actual_y': y_test,
            'y_predict': exp_predictions
            }
        })
    
    return pred_result['keras_log_y']

In [ ]:
from keras import Model
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from compile_keras import compile_keras_model
from eval_results import eval_results 
from stats_operations import safe_log, safe_exp

def train_nn_adamw(x_train, y_train, x_train_val, y_train_val, x_test, y_test):

    network = {
            'hidden_layers': [7, 7, 7, 7],
            'activation': 'relu',
            'optimizer': 'AdamW',
            'kernel_initializer': 'normal',
            'dropout': 0.1,
            'batch_size': 512,
            'model_type': 'mae',
            'int_layer': 30,
        }

    num_samples = x_train.shape[0]
    dimensions = x_train.shape[1]
    num_epochs = 500
    
    network['weight_decay'] = 0.005 * (network['batch_size'] / num_samples / num_epochs )**0.5

    model = compile_keras_model(network, dimensions)

    early_stopping = EarlyStopping(monitor='val_loss', patience=30)
    csv_logger = CSVLogger('./logs/nn-training-log.log')
    checkpointer = ModelCheckpoint(
        filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5)
    
    print('Training keras mae model...')

    # Reorder array - get array index
    array_index = np.arange(x_train.shape[0])

    # Reshuffle index
    np.random.shuffle(array_index)

    # Create array using new index
    x_shuffled_train = x_train[array_index]
    y_shuffled_train = safe_log(y_train[array_index])
    

    model.fit(x_shuffled_train,
              y_shuffled_train,
              validation_data=(x_train_val, safe_log(y_train_val)),
              epochs=num_epochs,
              batch_size=network['batch_size'],
              callbacks=[early_stopping, checkpointer, csv_logger, reduce_lr],
              verbose=True)

    model.load_weights('./weights/nn-weights.hdf5')

    print('Executing keras predictions...')

    log_y_predictions = model.predict(x_test)
    exp_predictions = safe_exp(log_y_predictions)

    pred_result = eval_results({'keras_log_y': {
            'actual_y': y_test,
            'y_predict': exp_predictions
            }
        })
    
    return pred_result['keras_log_y']

In [ ]:
def results_avg(results):
#     Loop through result dict and get keys
    keys = []
    for key in results[0].keys():
#         print(key)
        vals = []
        
        for result in results:
#             print(result[key])
            vals.append(result[key])

        print(key, ':', np.mean(vals))    
        


In [ ]:
for round in round_cols:
    msk = df_pp_train_x[round_name].values=='train'
    print(round, msk.sum())

In [ ]:
# Non tsfresh data
results = []

for round_name in round_cols:
    # Get x, y train / test (val) / holdout for this round   and remove the round columns  
    train_mask = df_pp_train_x[round_name].values=='train'
#     print(train_mask)
    train_x = remove_cols(df_pp_train_x[train_mask]).values
    train_y = all_y[train_mask]

    train_val_mask = df_pp_train_x[round_name].values=='test'
    train_val_x = remove_cols(df_pp_train_x[train_val_mask]).values
    train_val_y = all_y[train_val_mask]
    
    test_mask = df_pp_train_x[round_name].values=='holdout'
    test_x = remove_cols(df_pp_train_x[test_mask]).values
    test_y = all_y[test_mask]
          
    # Execute training with train / val / holdout
    round_result = train_nn_adamw(train_x, train_y, train_val_x, train_val_y, test_x, test_y) 
    print(round_name, 'result:', round_result)        
    results.append(round_result)
    
print('Average results')
print('-'*40)
results_avg(results)

In [ ]:
print('Average results')
print('-'*40)
results_avg(results)

In [ ]:
# tsfresh data - adamw
ts_results = []

round_cols = ['round_0', 'round_1', 'round_2', 'round_3', 'round_4']


for round_name in round_cols:
    # Get x, y train / test (val) / holdout for this round   and remove the round columns  
    train_mask = scaled_df_pp_augmented_train_x[round_name].values=='train'
#     print(train_mask)
    train_x = remove_cols(scaled_df_pp_augmented_train_x[train_mask]).values
    train_y = aug_y[train_mask]

    train_val_mask = scaled_df_pp_augmented_train_x[round_name].values=='test'
    train_val_x = remove_cols(scaled_df_pp_augmented_train_x[train_val_mask]).values
    train_val_y = aug_y[train_val_mask]
    
    test_mask = scaled_df_pp_augmented_train_x[round_name].values=='holdout'
    test_x = remove_cols(scaled_df_pp_augmented_train_x[test_mask]).values
    test_y = aug_y[test_mask]
          
    # Execute training with train / val / holdout
    round_result = train_nn_adamw(train_x, train_y, train_val_x, train_val_y, test_x, test_y) 
    print(round_name, 'result:', round_result)        
    ts_results.append(round_result)
    
print('Average tsfresh results')
print('-'*40)
results_avg(ts_results)

In [ ]:
print(aug_y.shape)
print(scaled_df_pp_augmented_train_x.shape)

In [ ]:
print('Results compare')
print('-'*40)
print('Average DL results')
print('-'*40)
results_avg(results)
print('Average ts fresh DL results')
print('-'*40)
results_avg(ts_results)

In [ ]:
for col in df_pp_augmented_train_x.columns:
    print(col, df_pp_augmented_train_x[col].dtype)
    
    if '__' in col:
        print(' -- tsfresh')

## Big stuff

In [ ]:
extract_feature_set = {
     'agg_autocorrelation': [
            {'f_agg': 'mean'},
            {'f_agg': 'median'},
            {'f_agg': 'var'}
     ], 
    'agg_linear_trend': [
      {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'max'},
      {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'min'},
      {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'mean'},
      {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'var'},
      {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'max'},
      {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'min'},
      {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'mean'},
      {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'var'},
      {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'max'},
      {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'min'},
      {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'mean'},
      {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'var'},
      {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'max'},
      {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'min'},
      {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'mean'},
      {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'var'},
      {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'max'},
      {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'min'},
      {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'mean'},
      {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'var'},
      {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'max'},
      {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'min'},
      {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'mean'},
      {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'var'},
      {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'max'},
      {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'min'},
      {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'mean'},
      {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'var'},
      {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'max'},
      {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'min'},
      {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'mean'},
      {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'var'},
      {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'max'},
      {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'min'},
      {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'mean'},
      {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'var'},
      {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'max'},
      {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'min'},
      {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'mean'},
      {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'var'},
      {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'max'},
      {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'min'},
      {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'mean'},
      {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'var'},
      {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'max'},
      {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'min'},
      {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'mean'},
      {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'var'}
    ],     
    'approximate_entropy': [
          {'m': 2, 'r': 0.1},
          {'m': 2, 'r': 0.3},
          {'m': 2, 'r': 0.5},
          {'m': 2, 'r': 0.7},
          {'m': 2, 'r': 0.9}
     ],
     'ar_coefficient': [
          {'coeff': 0, 'k': 10},
          {'coeff': 1, 'k': 10},
          {'coeff': 2, 'k': 10},
          {'coeff': 3, 'k': 10},
          {'coeff': 4, 'k': 10}
     ],
    'augmented_dickey_fuller': [
        {'attr': 'teststat'},
        {'attr': 'pvalue'},
        {'attr': 'usedlag'}
    ],    
    "autocorrelation": [
        {"lag": 0},
        {"lag": 1},
        {"lag": 2},
        {"lag": 3},
        {"lag": 4},
        {"lag": 5},
        {"lag": 6},
        {"lag": 7},
        {"lag": 8},
        {"lag": 9}
    ],
     'binned_entropy': [
         {'max_bins': 10}
     ],    
    'change_quantiles': [
        {'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
      {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
      {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
      {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
      {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.2, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.2, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.2, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.4, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.4, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.4, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.6, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.6, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': False, 'qh': 0.8, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': True, 'qh': 0.8, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8},
      {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.8},
      {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}
    ],
    "cid_ce": [
        {"normalize": True}, 
        {"normalize": False}
    ],
    'cwt_coefficients': [
      {'coeff': 0, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 0, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 0, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 0, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 1, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 1, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 1, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 1, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 2, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 2, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 2, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 2, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 3, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 3, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 3, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 3, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 4, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 4, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 4, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 4, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 5, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 5, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 5, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 5, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 6, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 6, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 6, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 6, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 7, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 7, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 7, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 7, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 8, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 8, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 8, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 8, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 9, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 9, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 9, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 9, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 10, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 10, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 10, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 10, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 11, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 11, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 11, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 11, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 12, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 12, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 12, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 12, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 13, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 13, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 13, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 13, 'w': 20, 'widths': (2, 5, 10, 20)},
      {'coeff': 14, 'w': 2, 'widths': (2, 5, 10, 20)},
      {'coeff': 14, 'w': 5, 'widths': (2, 5, 10, 20)},
      {'coeff': 14, 'w': 10, 'widths': (2, 5, 10, 20)},
      {'coeff': 14, 'w': 20, 'widths': (2, 5, 10, 20)}
     ],
    "energy_ratio_by_chunks": [
        {"num_segments" : 10, "segment_focus": 0},
        {"num_segments" : 10, "segment_focus": 1},
        {"num_segments" : 10, "segment_focus": 2},
        {"num_segments" : 10, "segment_focus": 3},
        {"num_segments" : 10, "segment_focus": 4},
        {"num_segments" : 10, "segment_focus": 5},
        {"num_segments" : 10, "segment_focus": 6},
        {"num_segments" : 10, "segment_focus": 7},
        {"num_segments" : 10, "segment_focus": 8},
        {"num_segments" : 10, "segment_focus": 9}
    ],    
    'fft_coefficient': [
          {'attr': 'real', 'coeff': 0},
          {'attr': 'real', 'coeff': 1},
          {'attr': 'real', 'coeff': 2},
          {'attr': 'real', 'coeff': 3},
          {'attr': 'real', 'coeff': 4},
          {'attr': 'real', 'coeff': 5},
          {'attr': 'real', 'coeff': 6},
          {'attr': 'real', 'coeff': 7},
          {'attr': 'real', 'coeff': 8},
          {'attr': 'real', 'coeff': 9},
          {'attr': 'real', 'coeff': 10},
          {'attr': 'real', 'coeff': 11},
          {'attr': 'real', 'coeff': 12},
          {'attr': 'real', 'coeff': 13},
          {'attr': 'real', 'coeff': 14},
          {'attr': 'real', 'coeff': 15},
          {'attr': 'real', 'coeff': 16},
          {'attr': 'real', 'coeff': 17},
          {'attr': 'real', 'coeff': 18},
          {'attr': 'real', 'coeff': 19},
          {'attr': 'real', 'coeff': 20},
          {'attr': 'real', 'coeff': 21},
          {'attr': 'real', 'coeff': 22},
          {'attr': 'real', 'coeff': 23},
          {'attr': 'real', 'coeff': 24},
          {'attr': 'real', 'coeff': 25},
          {'attr': 'real', 'coeff': 26},
          {'attr': 'real', 'coeff': 27},
          {'attr': 'real', 'coeff': 28},
          {'attr': 'real', 'coeff': 29},
          {'attr': 'real', 'coeff': 30},
          {'attr': 'real', 'coeff': 31},
          {'attr': 'real', 'coeff': 32},
          {'attr': 'real', 'coeff': 33},
          {'attr': 'real', 'coeff': 34},
          {'attr': 'real', 'coeff': 35},
          {'attr': 'real', 'coeff': 36},
          {'attr': 'real', 'coeff': 37},
          {'attr': 'real', 'coeff': 38},
          {'attr': 'real', 'coeff': 39},
          {'attr': 'real', 'coeff': 40},
          {'attr': 'real', 'coeff': 41},
          {'attr': 'real', 'coeff': 42},
          {'attr': 'real', 'coeff': 43},
          {'attr': 'real', 'coeff': 44},
          {'attr': 'real', 'coeff': 45},
          {'attr': 'real', 'coeff': 46},
          {'attr': 'real', 'coeff': 47},
          {'attr': 'real', 'coeff': 48},
          {'attr': 'real', 'coeff': 49},
          {'attr': 'real', 'coeff': 50},
          {'attr': 'real', 'coeff': 51},
          {'attr': 'real', 'coeff': 52},
          {'attr': 'real', 'coeff': 53},
          {'attr': 'real', 'coeff': 54},
          {'attr': 'real', 'coeff': 55},
          {'attr': 'real', 'coeff': 56},
          {'attr': 'real', 'coeff': 57},
          {'attr': 'real', 'coeff': 58},
          {'attr': 'real', 'coeff': 59},
          {'attr': 'real', 'coeff': 60},
          {'attr': 'real', 'coeff': 61},
          {'attr': 'real', 'coeff': 62},
          {'attr': 'real', 'coeff': 63},
          {'attr': 'real', 'coeff': 64},
          {'attr': 'real', 'coeff': 65},
          {'attr': 'real', 'coeff': 66},
          {'attr': 'real', 'coeff': 67},
          {'attr': 'real', 'coeff': 68},
          {'attr': 'real', 'coeff': 69},
          {'attr': 'real', 'coeff': 70},
          {'attr': 'real', 'coeff': 71},
          {'attr': 'real', 'coeff': 72},
          {'attr': 'real', 'coeff': 73},
          {'attr': 'real', 'coeff': 74},
          {'attr': 'real', 'coeff': 75},
          {'attr': 'real', 'coeff': 76},
          {'attr': 'real', 'coeff': 77},
          {'attr': 'real', 'coeff': 78},
          {'attr': 'real', 'coeff': 79},
          {'attr': 'real', 'coeff': 80},
          {'attr': 'real', 'coeff': 81},
          {'attr': 'real', 'coeff': 82},
          {'attr': 'real', 'coeff': 83},
          {'attr': 'real', 'coeff': 84},
          {'attr': 'real', 'coeff': 85},
          {'attr': 'real', 'coeff': 86},
          {'attr': 'real', 'coeff': 87},
          {'attr': 'real', 'coeff': 88},
          {'attr': 'real', 'coeff': 89},
          {'attr': 'real', 'coeff': 90},
          {'attr': 'real', 'coeff': 91},
          {'attr': 'real', 'coeff': 92},
          {'attr': 'real', 'coeff': 93},
          {'attr': 'real', 'coeff': 94},
          {'attr': 'real', 'coeff': 95},
          {'attr': 'real', 'coeff': 96},
          {'attr': 'real', 'coeff': 97},
          {'attr': 'real', 'coeff': 98},
          {'attr': 'real', 'coeff': 99},
          {'attr': 'imag', 'coeff': 0},
          {'attr': 'imag', 'coeff': 1},
          {'attr': 'imag', 'coeff': 2},
          {'attr': 'imag', 'coeff': 3},
          {'attr': 'imag', 'coeff': 4},
          {'attr': 'imag', 'coeff': 5},
          {'attr': 'imag', 'coeff': 6},
          {'attr': 'imag', 'coeff': 7},
          {'attr': 'imag', 'coeff': 8},
          {'attr': 'imag', 'coeff': 9},
          {'attr': 'imag', 'coeff': 10},
          {'attr': 'imag', 'coeff': 11},
          {'attr': 'imag', 'coeff': 12},
          {'attr': 'imag', 'coeff': 13},
          {'attr': 'imag', 'coeff': 14},
          {'attr': 'imag', 'coeff': 15},
          {'attr': 'imag', 'coeff': 16},
          {'attr': 'imag', 'coeff': 17},
          {'attr': 'imag', 'coeff': 18},
          {'attr': 'imag', 'coeff': 19},
          {'attr': 'imag', 'coeff': 20},
          {'attr': 'imag', 'coeff': 21},
          {'attr': 'imag', 'coeff': 22},
          {'attr': 'imag', 'coeff': 23},
          {'attr': 'imag', 'coeff': 24},
          {'attr': 'imag', 'coeff': 25},
          {'attr': 'imag', 'coeff': 26},
          {'attr': 'imag', 'coeff': 27},
          {'attr': 'imag', 'coeff': 28},
          {'attr': 'imag', 'coeff': 29},
          {'attr': 'imag', 'coeff': 30},
          {'attr': 'imag', 'coeff': 31},
          {'attr': 'imag', 'coeff': 32},
          {'attr': 'imag', 'coeff': 33},
          {'attr': 'imag', 'coeff': 34},
          {'attr': 'imag', 'coeff': 35},
          {'attr': 'imag', 'coeff': 36},
          {'attr': 'imag', 'coeff': 37},
          {'attr': 'imag', 'coeff': 38},
          {'attr': 'imag', 'coeff': 39},
          {'attr': 'imag', 'coeff': 40},
          {'attr': 'imag', 'coeff': 41},
          {'attr': 'imag', 'coeff': 42},
          {'attr': 'imag', 'coeff': 43},
          {'attr': 'imag', 'coeff': 44},
          {'attr': 'imag', 'coeff': 45},
          {'attr': 'imag', 'coeff': 46},
          {'attr': 'imag', 'coeff': 47},
          {'attr': 'imag', 'coeff': 48},
          {'attr': 'imag', 'coeff': 49},
          {'attr': 'imag', 'coeff': 50},
          {'attr': 'imag', 'coeff': 51},
          {'attr': 'imag', 'coeff': 52},
          {'attr': 'imag', 'coeff': 53},
          {'attr': 'imag', 'coeff': 54},
          {'attr': 'imag', 'coeff': 55},
          {'attr': 'imag', 'coeff': 56},
          {'attr': 'imag', 'coeff': 57},
          {'attr': 'imag', 'coeff': 58},
          {'attr': 'imag', 'coeff': 59},
          {'attr': 'imag', 'coeff': 60},
          {'attr': 'imag', 'coeff': 61},
          {'attr': 'imag', 'coeff': 62},
          {'attr': 'imag', 'coeff': 63},
          {'attr': 'imag', 'coeff': 64},
          {'attr': 'imag', 'coeff': 65},
          {'attr': 'imag', 'coeff': 66},
          {'attr': 'imag', 'coeff': 67},
          {'attr': 'imag', 'coeff': 68},
          {'attr': 'imag', 'coeff': 69},
          {'attr': 'imag', 'coeff': 70},
          {'attr': 'imag', 'coeff': 71},
          {'attr': 'imag', 'coeff': 72},
          {'attr': 'imag', 'coeff': 73},
          {'attr': 'imag', 'coeff': 74},
          {'attr': 'imag', 'coeff': 75},
          {'attr': 'imag', 'coeff': 76},
          {'attr': 'imag', 'coeff': 77},
          {'attr': 'imag', 'coeff': 78},
          {'attr': 'imag', 'coeff': 79},
          {'attr': 'imag', 'coeff': 80},
          {'attr': 'imag', 'coeff': 81},
          {'attr': 'imag', 'coeff': 82},
          {'attr': 'imag', 'coeff': 83},
          {'attr': 'imag', 'coeff': 84},
          {'attr': 'imag', 'coeff': 85},
          {'attr': 'imag', 'coeff': 86},
          {'attr': 'imag', 'coeff': 87},
          {'attr': 'imag', 'coeff': 88},
          {'attr': 'imag', 'coeff': 89},
          {'attr': 'imag', 'coeff': 90},
          {'attr': 'imag', 'coeff': 91},
          {'attr': 'imag', 'coeff': 92},
          {'attr': 'imag', 'coeff': 93},
          {'attr': 'imag', 'coeff': 94},
          {'attr': 'imag', 'coeff': 95},
          {'attr': 'imag', 'coeff': 96},
          {'attr': 'imag', 'coeff': 97},
          {'attr': 'imag', 'coeff': 98},
          {'attr': 'imag', 'coeff': 99}
     ],
    "fft_aggregated": [
        {"aggtype": "centroid"}, 
        {"aggtype": "variance"}, 
        {"aggtype": "skew"}, 
        {"aggtype": "kurtosis"}
    ],
    'first_location_of_maximum': None,
    'first_location_of_minimum': None,
    'kurtosis': None,
    'last_location_of_maximum': None,
    'last_location_of_minimum': None,
    'linear_trend': [
          {'attr': 'pvalue'},
          {'attr': 'rvalue'},
          {'attr': 'intercept'},
          {'attr': 'slope'},
          {'attr': 'stderr'}
    ],
    "partial_autocorrelation": [
        {"lag": 0},
        {"lag": 1},
        {"lag": 2},
        {"lag": 3},
        {"lag": 4},
        {"lag": 5},
        {"lag": 6},
        {"lag": 7},
        {"lag": 8},
        {"lag": 9}
    ],
    'skewness': None,
    'sample_entropy': None,    
    'spkt_welch_density': [
        {'coeff': 2}, 
        {'coeff': 5}, 
        {'coeff': 8}
    ],
}

In [ ]:
ALL_TSFRESH = ['value__abs_energy', 'value__absolute_sum_of_changes',
       'value__agg_autocorrelation__f_agg_"mean"',
       'value__agg_autocorrelation__f_agg_"median"',
       'value__agg_autocorrelation__f_agg_"var"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"slope"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"slope"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"slope"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"stderr"',
       'value__approximate_entropy__m_2__r_0.1',
       'value__approximate_entropy__m_2__r_0.3',
       'value__approximate_entropy__m_2__r_0.5',
       'value__approximate_entropy__m_2__r_0.7',
       'value__approximate_entropy__m_2__r_0.9',
       'value__ar_coefficient__k_10__coeff_0',
       'value__ar_coefficient__k_10__coeff_1',
       'value__ar_coefficient__k_10__coeff_2',
       'value__ar_coefficient__k_10__coeff_3',
       'value__ar_coefficient__k_10__coeff_4',
       'value__augmented_dickey_fuller__attr_"pvalue"',
       'value__augmented_dickey_fuller__attr_"teststat"',
       'value__augmented_dickey_fuller__attr_"usedlag"',
       'value__autocorrelation__lag_0', 'value__autocorrelation__lag_1',
       'value__autocorrelation__lag_2', 'value__autocorrelation__lag_3',
       'value__autocorrelation__lag_4', 'value__autocorrelation__lag_5',
       'value__autocorrelation__lag_6', 'value__autocorrelation__lag_7',
       'value__autocorrelation__lag_8', 'value__autocorrelation__lag_9',
       'value__binned_entropy__max_bins_10', 'value__c3__lag_1',
       'value__c3__lag_2', 'value__c3__lag_3',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',
       'value__cid_ce__normalize_False', 'value__cid_ce__normalize_True',
       'value__count_above_mean', 'value__count_below_mean',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_1',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_2',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_3',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_4',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_5',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_6',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_7',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_9',
       'value__fft_aggregated__aggtype_"centroid"',
       'value__fft_aggregated__aggtype_"kurtosis"',
       'value__fft_aggregated__aggtype_"skew"',
       'value__fft_aggregated__aggtype_"variance"',
       'value__fft_coefficient__coeff_0__attr_"abs"',
       'value__fft_coefficient__coeff_0__attr_"angle"',
       'value__fft_coefficient__coeff_0__attr_"imag"',
       'value__fft_coefficient__coeff_0__attr_"real"',
       'value__fft_coefficient__coeff_10__attr_"abs"',
       'value__fft_coefficient__coeff_10__attr_"angle"',
       'value__fft_coefficient__coeff_10__attr_"imag"',
       'value__fft_coefficient__coeff_10__attr_"real"',
       'value__fft_coefficient__coeff_11__attr_"abs"',
       'value__fft_coefficient__coeff_11__attr_"angle"',
       'value__fft_coefficient__coeff_11__attr_"imag"',
       'value__fft_coefficient__coeff_11__attr_"real"',
       'value__fft_coefficient__coeff_12__attr_"abs"',
       'value__fft_coefficient__coeff_12__attr_"angle"',
       'value__fft_coefficient__coeff_12__attr_"imag"',
       'value__fft_coefficient__coeff_12__attr_"real"',
       'value__fft_coefficient__coeff_13__attr_"abs"',
       'value__fft_coefficient__coeff_13__attr_"angle"',
       'value__fft_coefficient__coeff_13__attr_"imag"',
       'value__fft_coefficient__coeff_13__attr_"real"',
       'value__fft_coefficient__coeff_14__attr_"abs"',
       'value__fft_coefficient__coeff_14__attr_"angle"',
       'value__fft_coefficient__coeff_14__attr_"imag"',
       'value__fft_coefficient__coeff_14__attr_"real"',
       'value__fft_coefficient__coeff_15__attr_"abs"',
       'value__fft_coefficient__coeff_15__attr_"angle"',
       'value__fft_coefficient__coeff_15__attr_"imag"',
       'value__fft_coefficient__coeff_15__attr_"real"',
       'value__fft_coefficient__coeff_16__attr_"abs"',
       'value__fft_coefficient__coeff_16__attr_"angle"',
       'value__fft_coefficient__coeff_16__attr_"imag"',
       'value__fft_coefficient__coeff_16__attr_"real"',
       'value__fft_coefficient__coeff_17__attr_"abs"',
       'value__fft_coefficient__coeff_17__attr_"angle"',
       'value__fft_coefficient__coeff_17__attr_"imag"',
       'value__fft_coefficient__coeff_17__attr_"real"',
       'value__fft_coefficient__coeff_18__attr_"abs"',
       'value__fft_coefficient__coeff_18__attr_"angle"',
       'value__fft_coefficient__coeff_18__attr_"imag"',
       'value__fft_coefficient__coeff_18__attr_"real"',
       'value__fft_coefficient__coeff_19__attr_"abs"',
       'value__fft_coefficient__coeff_19__attr_"angle"',
       'value__fft_coefficient__coeff_19__attr_"imag"',
       'value__fft_coefficient__coeff_19__attr_"real"',
       'value__fft_coefficient__coeff_1__attr_"abs"',
       'value__fft_coefficient__coeff_1__attr_"angle"',
       'value__fft_coefficient__coeff_1__attr_"imag"',
       'value__fft_coefficient__coeff_1__attr_"real"',
       'value__fft_coefficient__coeff_20__attr_"abs"',
       'value__fft_coefficient__coeff_20__attr_"angle"',
       'value__fft_coefficient__coeff_20__attr_"imag"',
       'value__fft_coefficient__coeff_20__attr_"real"',
       'value__fft_coefficient__coeff_21__attr_"abs"',
       'value__fft_coefficient__coeff_21__attr_"angle"',
       'value__fft_coefficient__coeff_21__attr_"imag"',
       'value__fft_coefficient__coeff_21__attr_"real"',
       'value__fft_coefficient__coeff_22__attr_"abs"',
       'value__fft_coefficient__coeff_22__attr_"angle"',
       'value__fft_coefficient__coeff_22__attr_"imag"',
       'value__fft_coefficient__coeff_22__attr_"real"',
       'value__fft_coefficient__coeff_23__attr_"abs"',
       'value__fft_coefficient__coeff_23__attr_"angle"',
       'value__fft_coefficient__coeff_23__attr_"imag"',
       'value__fft_coefficient__coeff_23__attr_"real"',
       'value__fft_coefficient__coeff_24__attr_"abs"',
       'value__fft_coefficient__coeff_24__attr_"angle"',
       'value__fft_coefficient__coeff_24__attr_"imag"',
       'value__fft_coefficient__coeff_24__attr_"real"',
       'value__fft_coefficient__coeff_25__attr_"abs"',
       'value__fft_coefficient__coeff_25__attr_"angle"',
       'value__fft_coefficient__coeff_25__attr_"imag"',
       'value__fft_coefficient__coeff_25__attr_"real"',
       'value__fft_coefficient__coeff_26__attr_"abs"',
       'value__fft_coefficient__coeff_26__attr_"angle"',
       'value__fft_coefficient__coeff_26__attr_"imag"',
       'value__fft_coefficient__coeff_26__attr_"real"',
       'value__fft_coefficient__coeff_27__attr_"abs"',
       'value__fft_coefficient__coeff_27__attr_"angle"',
       'value__fft_coefficient__coeff_27__attr_"imag"',
       'value__fft_coefficient__coeff_27__attr_"real"',
       'value__fft_coefficient__coeff_28__attr_"abs"',
       'value__fft_coefficient__coeff_28__attr_"angle"',
       'value__fft_coefficient__coeff_28__attr_"imag"',
       'value__fft_coefficient__coeff_28__attr_"real"',
       'value__fft_coefficient__coeff_29__attr_"abs"',
       'value__fft_coefficient__coeff_29__attr_"angle"',
       'value__fft_coefficient__coeff_29__attr_"imag"',
       'value__fft_coefficient__coeff_29__attr_"real"',
       'value__fft_coefficient__coeff_2__attr_"abs"',
       'value__fft_coefficient__coeff_2__attr_"angle"',
       'value__fft_coefficient__coeff_2__attr_"imag"',
       'value__fft_coefficient__coeff_2__attr_"real"',
       'value__fft_coefficient__coeff_30__attr_"abs"',
       'value__fft_coefficient__coeff_30__attr_"angle"',
       'value__fft_coefficient__coeff_30__attr_"imag"',
       'value__fft_coefficient__coeff_30__attr_"real"',
       'value__fft_coefficient__coeff_31__attr_"abs"',
       'value__fft_coefficient__coeff_31__attr_"angle"',
       'value__fft_coefficient__coeff_31__attr_"imag"',
       'value__fft_coefficient__coeff_31__attr_"real"',
       'value__fft_coefficient__coeff_32__attr_"abs"',
       'value__fft_coefficient__coeff_32__attr_"angle"',
       'value__fft_coefficient__coeff_32__attr_"imag"',
       'value__fft_coefficient__coeff_32__attr_"real"',
       'value__fft_coefficient__coeff_33__attr_"abs"',
       'value__fft_coefficient__coeff_33__attr_"angle"',
       'value__fft_coefficient__coeff_33__attr_"imag"',
       'value__fft_coefficient__coeff_33__attr_"real"',
       'value__fft_coefficient__coeff_34__attr_"abs"',
       'value__fft_coefficient__coeff_34__attr_"angle"',
       'value__fft_coefficient__coeff_34__attr_"imag"',
       'value__fft_coefficient__coeff_34__attr_"real"',
       'value__fft_coefficient__coeff_35__attr_"abs"',
       'value__fft_coefficient__coeff_35__attr_"angle"',
       'value__fft_coefficient__coeff_35__attr_"imag"',
       'value__fft_coefficient__coeff_35__attr_"real"',
       'value__fft_coefficient__coeff_36__attr_"abs"',
       'value__fft_coefficient__coeff_36__attr_"angle"',
       'value__fft_coefficient__coeff_36__attr_"imag"',
       'value__fft_coefficient__coeff_36__attr_"real"',
       'value__fft_coefficient__coeff_37__attr_"abs"',
       'value__fft_coefficient__coeff_37__attr_"angle"',
       'value__fft_coefficient__coeff_37__attr_"imag"',
       'value__fft_coefficient__coeff_37__attr_"real"',
       'value__fft_coefficient__coeff_38__attr_"abs"',
       'value__fft_coefficient__coeff_38__attr_"angle"',
       'value__fft_coefficient__coeff_38__attr_"imag"',
       'value__fft_coefficient__coeff_38__attr_"real"',
       'value__fft_coefficient__coeff_39__attr_"abs"',
       'value__fft_coefficient__coeff_39__attr_"angle"',
       'value__fft_coefficient__coeff_39__attr_"imag"',
       'value__fft_coefficient__coeff_39__attr_"real"',
       'value__fft_coefficient__coeff_3__attr_"abs"',
       'value__fft_coefficient__coeff_3__attr_"angle"',
       'value__fft_coefficient__coeff_3__attr_"imag"',
       'value__fft_coefficient__coeff_3__attr_"real"',
       'value__fft_coefficient__coeff_40__attr_"abs"',
       'value__fft_coefficient__coeff_40__attr_"angle"',
       'value__fft_coefficient__coeff_40__attr_"imag"',
       'value__fft_coefficient__coeff_40__attr_"real"',
       'value__fft_coefficient__coeff_41__attr_"abs"',
       'value__fft_coefficient__coeff_41__attr_"angle"',
       'value__fft_coefficient__coeff_41__attr_"imag"',
       'value__fft_coefficient__coeff_41__attr_"real"',
       'value__fft_coefficient__coeff_42__attr_"abs"',
       'value__fft_coefficient__coeff_42__attr_"angle"',
       'value__fft_coefficient__coeff_42__attr_"imag"',
       'value__fft_coefficient__coeff_42__attr_"real"',
       'value__fft_coefficient__coeff_43__attr_"abs"',
       'value__fft_coefficient__coeff_43__attr_"angle"',
       'value__fft_coefficient__coeff_43__attr_"imag"',
       'value__fft_coefficient__coeff_43__attr_"real"',
       'value__fft_coefficient__coeff_44__attr_"abs"',
       'value__fft_coefficient__coeff_44__attr_"angle"',
       'value__fft_coefficient__coeff_44__attr_"imag"',
       'value__fft_coefficient__coeff_44__attr_"real"',
       'value__fft_coefficient__coeff_45__attr_"abs"',
       'value__fft_coefficient__coeff_45__attr_"angle"',
       'value__fft_coefficient__coeff_45__attr_"imag"',
       'value__fft_coefficient__coeff_45__attr_"real"',
       'value__fft_coefficient__coeff_46__attr_"abs"',
       'value__fft_coefficient__coeff_46__attr_"angle"',
       'value__fft_coefficient__coeff_46__attr_"imag"',
       'value__fft_coefficient__coeff_46__attr_"real"',
       'value__fft_coefficient__coeff_47__attr_"abs"',
       'value__fft_coefficient__coeff_47__attr_"angle"',
       'value__fft_coefficient__coeff_47__attr_"imag"',
       'value__fft_coefficient__coeff_47__attr_"real"',
       'value__fft_coefficient__coeff_48__attr_"abs"',
       'value__fft_coefficient__coeff_48__attr_"angle"',
       'value__fft_coefficient__coeff_48__attr_"imag"',
       'value__fft_coefficient__coeff_48__attr_"real"',
       'value__fft_coefficient__coeff_49__attr_"abs"',
       'value__fft_coefficient__coeff_49__attr_"angle"',
       'value__fft_coefficient__coeff_49__attr_"imag"',
       'value__fft_coefficient__coeff_49__attr_"real"',
       'value__fft_coefficient__coeff_4__attr_"abs"',
       'value__fft_coefficient__coeff_4__attr_"angle"',
       'value__fft_coefficient__coeff_4__attr_"imag"',
       'value__fft_coefficient__coeff_4__attr_"real"',
       'value__fft_coefficient__coeff_50__attr_"abs"',
       'value__fft_coefficient__coeff_50__attr_"angle"',
       'value__fft_coefficient__coeff_50__attr_"imag"',
       'value__fft_coefficient__coeff_50__attr_"real"',
       'value__fft_coefficient__coeff_51__attr_"abs"',
       'value__fft_coefficient__coeff_51__attr_"angle"',
       'value__fft_coefficient__coeff_51__attr_"imag"',
       'value__fft_coefficient__coeff_51__attr_"real"',
       'value__fft_coefficient__coeff_52__attr_"abs"',
       'value__fft_coefficient__coeff_52__attr_"angle"',
       'value__fft_coefficient__coeff_52__attr_"imag"',
       'value__fft_coefficient__coeff_52__attr_"real"',
       'value__fft_coefficient__coeff_53__attr_"abs"',
       'value__fft_coefficient__coeff_53__attr_"angle"',
       'value__fft_coefficient__coeff_53__attr_"imag"',
       'value__fft_coefficient__coeff_53__attr_"real"',
       'value__fft_coefficient__coeff_54__attr_"abs"',
       'value__fft_coefficient__coeff_54__attr_"angle"',
       'value__fft_coefficient__coeff_54__attr_"imag"',
       'value__fft_coefficient__coeff_54__attr_"real"',
       'value__fft_coefficient__coeff_55__attr_"abs"',
       'value__fft_coefficient__coeff_55__attr_"angle"',
       'value__fft_coefficient__coeff_55__attr_"imag"',
       'value__fft_coefficient__coeff_55__attr_"real"',
       'value__fft_coefficient__coeff_56__attr_"abs"',
       'value__fft_coefficient__coeff_56__attr_"angle"',
       'value__fft_coefficient__coeff_56__attr_"imag"',
       'value__fft_coefficient__coeff_56__attr_"real"',
       'value__fft_coefficient__coeff_57__attr_"abs"',
       'value__fft_coefficient__coeff_57__attr_"angle"',
       'value__fft_coefficient__coeff_57__attr_"imag"',
       'value__fft_coefficient__coeff_57__attr_"real"',
       'value__fft_coefficient__coeff_58__attr_"abs"',
       'value__fft_coefficient__coeff_58__attr_"angle"',
       'value__fft_coefficient__coeff_58__attr_"imag"',
       'value__fft_coefficient__coeff_58__attr_"real"',
       'value__fft_coefficient__coeff_59__attr_"abs"',
       'value__fft_coefficient__coeff_59__attr_"angle"',
       'value__fft_coefficient__coeff_59__attr_"imag"',
       'value__fft_coefficient__coeff_59__attr_"real"',
       'value__fft_coefficient__coeff_5__attr_"abs"',
       'value__fft_coefficient__coeff_5__attr_"angle"',
       'value__fft_coefficient__coeff_5__attr_"imag"',
       'value__fft_coefficient__coeff_5__attr_"real"',
       'value__fft_coefficient__coeff_60__attr_"abs"',
       'value__fft_coefficient__coeff_60__attr_"angle"',
       'value__fft_coefficient__coeff_60__attr_"imag"',
       'value__fft_coefficient__coeff_60__attr_"real"',
       'value__fft_coefficient__coeff_61__attr_"abs"',
       'value__fft_coefficient__coeff_61__attr_"angle"',
       'value__fft_coefficient__coeff_61__attr_"imag"',
       'value__fft_coefficient__coeff_61__attr_"real"',
       'value__fft_coefficient__coeff_62__attr_"abs"',
       'value__fft_coefficient__coeff_62__attr_"angle"',
       'value__fft_coefficient__coeff_62__attr_"imag"',
       'value__fft_coefficient__coeff_62__attr_"real"',
       'value__fft_coefficient__coeff_63__attr_"abs"',
       'value__fft_coefficient__coeff_63__attr_"angle"',
       'value__fft_coefficient__coeff_63__attr_"imag"',
       'value__fft_coefficient__coeff_63__attr_"real"',
       'value__fft_coefficient__coeff_64__attr_"abs"',
       'value__fft_coefficient__coeff_64__attr_"angle"',
       'value__fft_coefficient__coeff_64__attr_"imag"',
       'value__fft_coefficient__coeff_64__attr_"real"',
       'value__fft_coefficient__coeff_65__attr_"abs"',
       'value__fft_coefficient__coeff_65__attr_"angle"',
       'value__fft_coefficient__coeff_65__attr_"imag"',
       'value__fft_coefficient__coeff_65__attr_"real"',
       'value__fft_coefficient__coeff_66__attr_"abs"',
       'value__fft_coefficient__coeff_66__attr_"angle"',
       'value__fft_coefficient__coeff_66__attr_"imag"',
       'value__fft_coefficient__coeff_66__attr_"real"',
       'value__fft_coefficient__coeff_67__attr_"abs"',
       'value__fft_coefficient__coeff_67__attr_"angle"',
       'value__fft_coefficient__coeff_67__attr_"imag"',
       'value__fft_coefficient__coeff_67__attr_"real"',
       'value__fft_coefficient__coeff_68__attr_"abs"',
       'value__fft_coefficient__coeff_68__attr_"angle"',
       'value__fft_coefficient__coeff_68__attr_"imag"',
       'value__fft_coefficient__coeff_68__attr_"real"',
       'value__fft_coefficient__coeff_69__attr_"abs"',
       'value__fft_coefficient__coeff_69__attr_"angle"',
       'value__fft_coefficient__coeff_69__attr_"imag"',
       'value__fft_coefficient__coeff_69__attr_"real"',
       'value__fft_coefficient__coeff_6__attr_"abs"',
       'value__fft_coefficient__coeff_6__attr_"angle"',
       'value__fft_coefficient__coeff_6__attr_"imag"',
       'value__fft_coefficient__coeff_6__attr_"real"',
       'value__fft_coefficient__coeff_70__attr_"abs"',
       'value__fft_coefficient__coeff_70__attr_"angle"',
       'value__fft_coefficient__coeff_70__attr_"imag"',
       'value__fft_coefficient__coeff_70__attr_"real"',
       'value__fft_coefficient__coeff_71__attr_"abs"',
       'value__fft_coefficient__coeff_71__attr_"angle"',
       'value__fft_coefficient__coeff_71__attr_"imag"',
       'value__fft_coefficient__coeff_71__attr_"real"',
       'value__fft_coefficient__coeff_72__attr_"abs"',
       'value__fft_coefficient__coeff_72__attr_"angle"',
       'value__fft_coefficient__coeff_72__attr_"imag"',
       'value__fft_coefficient__coeff_72__attr_"real"',
       'value__fft_coefficient__coeff_73__attr_"abs"',
       'value__fft_coefficient__coeff_73__attr_"angle"',
       'value__fft_coefficient__coeff_73__attr_"imag"',
       'value__fft_coefficient__coeff_73__attr_"real"',
       'value__fft_coefficient__coeff_74__attr_"abs"',
       'value__fft_coefficient__coeff_74__attr_"angle"',
       'value__fft_coefficient__coeff_74__attr_"imag"',
       'value__fft_coefficient__coeff_74__attr_"real"',
       'value__fft_coefficient__coeff_75__attr_"abs"',
       'value__fft_coefficient__coeff_75__attr_"angle"',
       'value__fft_coefficient__coeff_75__attr_"imag"',
       'value__fft_coefficient__coeff_75__attr_"real"',
       'value__fft_coefficient__coeff_76__attr_"abs"',
       'value__fft_coefficient__coeff_76__attr_"angle"',
       'value__fft_coefficient__coeff_76__attr_"imag"',
       'value__fft_coefficient__coeff_76__attr_"real"',
       'value__fft_coefficient__coeff_77__attr_"abs"',
       'value__fft_coefficient__coeff_77__attr_"angle"',
       'value__fft_coefficient__coeff_77__attr_"imag"',
       'value__fft_coefficient__coeff_77__attr_"real"',
       'value__fft_coefficient__coeff_78__attr_"abs"',
       'value__fft_coefficient__coeff_78__attr_"angle"',
       'value__fft_coefficient__coeff_78__attr_"imag"',
       'value__fft_coefficient__coeff_78__attr_"real"',
       'value__fft_coefficient__coeff_79__attr_"abs"',
       'value__fft_coefficient__coeff_79__attr_"angle"',
       'value__fft_coefficient__coeff_79__attr_"imag"',
       'value__fft_coefficient__coeff_79__attr_"real"',
       'value__fft_coefficient__coeff_7__attr_"abs"',
       'value__fft_coefficient__coeff_7__attr_"angle"',
       'value__fft_coefficient__coeff_7__attr_"imag"',
       'value__fft_coefficient__coeff_7__attr_"real"',
       'value__fft_coefficient__coeff_80__attr_"abs"',
       'value__fft_coefficient__coeff_80__attr_"angle"',
       'value__fft_coefficient__coeff_80__attr_"imag"',
       'value__fft_coefficient__coeff_80__attr_"real"',
       'value__fft_coefficient__coeff_81__attr_"abs"',
       'value__fft_coefficient__coeff_81__attr_"angle"',
       'value__fft_coefficient__coeff_81__attr_"imag"',
       'value__fft_coefficient__coeff_81__attr_"real"',
       'value__fft_coefficient__coeff_82__attr_"abs"',
       'value__fft_coefficient__coeff_82__attr_"angle"',
       'value__fft_coefficient__coeff_82__attr_"imag"',
       'value__fft_coefficient__coeff_82__attr_"real"',
       'value__fft_coefficient__coeff_83__attr_"abs"',
       'value__fft_coefficient__coeff_83__attr_"angle"',
       'value__fft_coefficient__coeff_83__attr_"imag"',
       'value__fft_coefficient__coeff_83__attr_"real"',
       'value__fft_coefficient__coeff_84__attr_"abs"',
       'value__fft_coefficient__coeff_84__attr_"angle"',
       'value__fft_coefficient__coeff_84__attr_"imag"',
       'value__fft_coefficient__coeff_84__attr_"real"',
       'value__fft_coefficient__coeff_85__attr_"abs"',
       'value__fft_coefficient__coeff_85__attr_"angle"',
       'value__fft_coefficient__coeff_85__attr_"imag"',
       'value__fft_coefficient__coeff_85__attr_"real"',
       'value__fft_coefficient__coeff_86__attr_"abs"',
       'value__fft_coefficient__coeff_86__attr_"angle"',
       'value__fft_coefficient__coeff_86__attr_"imag"',
       'value__fft_coefficient__coeff_86__attr_"real"',
       'value__fft_coefficient__coeff_87__attr_"abs"',
       'value__fft_coefficient__coeff_87__attr_"angle"',
       'value__fft_coefficient__coeff_87__attr_"imag"',
       'value__fft_coefficient__coeff_87__attr_"real"',
       'value__fft_coefficient__coeff_88__attr_"abs"',
       'value__fft_coefficient__coeff_88__attr_"angle"',
       'value__fft_coefficient__coeff_88__attr_"imag"',
       'value__fft_coefficient__coeff_88__attr_"real"',
       'value__fft_coefficient__coeff_89__attr_"abs"',
       'value__fft_coefficient__coeff_89__attr_"angle"',
       'value__fft_coefficient__coeff_89__attr_"imag"',
       'value__fft_coefficient__coeff_89__attr_"real"',
       'value__fft_coefficient__coeff_8__attr_"abs"',
       'value__fft_coefficient__coeff_8__attr_"angle"',
       'value__fft_coefficient__coeff_8__attr_"imag"',
       'value__fft_coefficient__coeff_8__attr_"real"',
       'value__fft_coefficient__coeff_90__attr_"abs"',
       'value__fft_coefficient__coeff_90__attr_"angle"',
       'value__fft_coefficient__coeff_90__attr_"imag"',
       'value__fft_coefficient__coeff_90__attr_"real"',
       'value__fft_coefficient__coeff_91__attr_"abs"',
       'value__fft_coefficient__coeff_91__attr_"angle"',
       'value__fft_coefficient__coeff_91__attr_"imag"',
       'value__fft_coefficient__coeff_91__attr_"real"',
       'value__fft_coefficient__coeff_92__attr_"abs"',
       'value__fft_coefficient__coeff_92__attr_"angle"',
       'value__fft_coefficient__coeff_92__attr_"imag"',
       'value__fft_coefficient__coeff_92__attr_"real"',
       'value__fft_coefficient__coeff_93__attr_"abs"',
       'value__fft_coefficient__coeff_93__attr_"angle"',
       'value__fft_coefficient__coeff_93__attr_"imag"',
       'value__fft_coefficient__coeff_93__attr_"real"',
       'value__fft_coefficient__coeff_94__attr_"abs"',
       'value__fft_coefficient__coeff_94__attr_"angle"',
       'value__fft_coefficient__coeff_94__attr_"imag"',
       'value__fft_coefficient__coeff_94__attr_"real"',
       'value__fft_coefficient__coeff_95__attr_"abs"',
       'value__fft_coefficient__coeff_95__attr_"angle"',
       'value__fft_coefficient__coeff_95__attr_"imag"',
       'value__fft_coefficient__coeff_95__attr_"real"',
       'value__fft_coefficient__coeff_96__attr_"abs"',
       'value__fft_coefficient__coeff_96__attr_"angle"',
       'value__fft_coefficient__coeff_96__attr_"imag"',
       'value__fft_coefficient__coeff_96__attr_"real"',
       'value__fft_coefficient__coeff_97__attr_"abs"',
       'value__fft_coefficient__coeff_97__attr_"angle"',
       'value__fft_coefficient__coeff_97__attr_"imag"',
       'value__fft_coefficient__coeff_97__attr_"real"',
       'value__fft_coefficient__coeff_98__attr_"abs"',
       'value__fft_coefficient__coeff_98__attr_"angle"',
       'value__fft_coefficient__coeff_98__attr_"imag"',
       'value__fft_coefficient__coeff_98__attr_"real"',
       'value__fft_coefficient__coeff_99__attr_"abs"',
       'value__fft_coefficient__coeff_99__attr_"angle"',
       'value__fft_coefficient__coeff_99__attr_"imag"',
       'value__fft_coefficient__coeff_99__attr_"real"',
       'value__fft_coefficient__coeff_9__attr_"abs"',
       'value__fft_coefficient__coeff_9__attr_"angle"',
       'value__fft_coefficient__coeff_9__attr_"imag"',
       'value__fft_coefficient__coeff_9__attr_"real"',
       'value__first_location_of_maximum',
       'value__first_location_of_minimum',
       'value__friedrich_coefficients__m_3__r_30__coeff_0',
       'value__friedrich_coefficients__m_3__r_30__coeff_1',
       'value__friedrich_coefficients__m_3__r_30__coeff_2',
       'value__friedrich_coefficients__m_3__r_30__coeff_3',
       'value__has_duplicate', 'value__has_duplicate_max',
       'value__has_duplicate_min', 'value__index_mass_quantile__q_0.1',
       'value__index_mass_quantile__q_0.2',
       'value__index_mass_quantile__q_0.3',
       'value__index_mass_quantile__q_0.4',
       'value__index_mass_quantile__q_0.6',
       'value__index_mass_quantile__q_0.7',
       'value__index_mass_quantile__q_0.8',
       'value__index_mass_quantile__q_0.9', 'value__kurtosis',
       'value__large_standard_deviation__r_0.05',
       'value__large_standard_deviation__r_0.1',
       'value__large_standard_deviation__r_0.15000000000000002',
       'value__large_standard_deviation__r_0.2',
       'value__large_standard_deviation__r_0.25',
       'value__large_standard_deviation__r_0.30000000000000004',
       'value__large_standard_deviation__r_0.35000000000000003',
       'value__large_standard_deviation__r_0.4',
       'value__large_standard_deviation__r_0.45',
       'value__large_standard_deviation__r_0.5',
       'value__large_standard_deviation__r_0.55',
       'value__large_standard_deviation__r_0.6000000000000001',
       'value__large_standard_deviation__r_0.65',
       'value__large_standard_deviation__r_0.7000000000000001',
       'value__large_standard_deviation__r_0.75',
       'value__large_standard_deviation__r_0.8',
       'value__large_standard_deviation__r_0.8500000000000001',
       'value__large_standard_deviation__r_0.9',
       'value__large_standard_deviation__r_0.9500000000000001',
       'value__last_location_of_maximum',
       'value__last_location_of_minimum', 'value__length',
       'value__linear_trend__attr_"intercept"',
       'value__linear_trend__attr_"pvalue"',
       'value__linear_trend__attr_"rvalue"',
       'value__linear_trend__attr_"slope"',
       'value__linear_trend__attr_"stderr"',
       'value__longest_strike_above_mean',
       'value__longest_strike_below_mean',
       'value__max_langevin_fixed_point__m_3__r_30', 'value__maximum',
       'value__mean', 'value__mean_abs_change', 'value__mean_change',
       'value__mean_second_derivative_central', 'value__median',
       'value__minimum', 'value__number_crossing_m__m_-1',
       'value__number_crossing_m__m_0', 'value__number_crossing_m__m_1',
       'value__number_cwt_peaks__n_1', 'value__number_cwt_peaks__n_5',
       'value__number_peaks__n_1', 'value__number_peaks__n_10',
       'value__number_peaks__n_3', 'value__number_peaks__n_5',
       'value__number_peaks__n_50',
       'value__partial_autocorrelation__lag_0',
       'value__partial_autocorrelation__lag_1',
       'value__partial_autocorrelation__lag_2',
       'value__partial_autocorrelation__lag_3',
       'value__partial_autocorrelation__lag_4',
       'value__partial_autocorrelation__lag_5',
       'value__partial_autocorrelation__lag_6',
       'value__partial_autocorrelation__lag_7',
       'value__partial_autocorrelation__lag_8',
       'value__partial_autocorrelation__lag_9',
       'value__percentage_of_reoccurring_datapoints_to_all_datapoints',
       'value__percentage_of_reoccurring_values_to_all_values',
       'value__quantile__q_0.1', 'value__quantile__q_0.2',
       'value__quantile__q_0.3', 'value__quantile__q_0.4',
       'value__quantile__q_0.6', 'value__quantile__q_0.7',
       'value__quantile__q_0.8', 'value__quantile__q_0.9',
       'value__range_count__max_1__min_-1',
       'value__ratio_beyond_r_sigma__r_0.5',
       'value__ratio_beyond_r_sigma__r_1',
       'value__ratio_beyond_r_sigma__r_1.5',
       'value__ratio_beyond_r_sigma__r_10',
       'value__ratio_beyond_r_sigma__r_2',
       'value__ratio_beyond_r_sigma__r_2.5',
       'value__ratio_beyond_r_sigma__r_3',
       'value__ratio_beyond_r_sigma__r_5',
       'value__ratio_beyond_r_sigma__r_6',
       'value__ratio_beyond_r_sigma__r_7',
       'value__ratio_value_number_to_time_series_length',
       'value__sample_entropy', 'value__skewness',
       'value__spkt_welch_density__coeff_2',
       'value__spkt_welch_density__coeff_5',
       'value__spkt_welch_density__coeff_8', 'value__standard_deviation',
       'value__sum_of_reoccurring_data_points',
       'value__sum_of_reoccurring_values', 'value__sum_values',
       'value__symmetry_looking__r_0.0',
       'value__symmetry_looking__r_0.05',
       'value__symmetry_looking__r_0.1',
       'value__symmetry_looking__r_0.15000000000000002',
       'value__symmetry_looking__r_0.2',
       'value__symmetry_looking__r_0.25',
       'value__symmetry_looking__r_0.30000000000000004',
       'value__symmetry_looking__r_0.35000000000000003',
       'value__symmetry_looking__r_0.4',
       'value__symmetry_looking__r_0.45',
       'value__symmetry_looking__r_0.5',
       'value__symmetry_looking__r_0.55',
       'value__symmetry_looking__r_0.6000000000000001',
       'value__symmetry_looking__r_0.65',
       'value__symmetry_looking__r_0.7000000000000001',
       'value__symmetry_looking__r_0.75',
       'value__symmetry_looking__r_0.8',
       'value__symmetry_looking__r_0.8500000000000001',
       'value__symmetry_looking__r_0.9',
       'value__symmetry_looking__r_0.9500000000000001',
       'value__time_reversal_asymmetry_statistic__lag_1',
       'value__time_reversal_asymmetry_statistic__lag_2',
       'value__time_reversal_asymmetry_statistic__lag_3',
       'value__value_count__value_-inf', 'value__value_count__value_0',
       'value__value_count__value_1', 'value__value_count__value_inf',
       'value__value_count__value_nan', 'value__variance',
       'value__variance_larger_than_standard_deviation']

In [ ]:
TSFRESH_RELEVANT = ['value__abs_energy', 'value__absolute_sum_of_changes',
       'value__agg_autocorrelation__f_agg_"mean"',
       'value__agg_autocorrelation__f_agg_"median"',
       'value__agg_autocorrelation__f_agg_"var"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"max"__chunk_len_5__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"mean"__chunk_len_5__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"min"__chunk_len_5__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"slope"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_10__attr_"stderr"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"intercept"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"rvalue"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"slope"',
       'value__agg_linear_trend__f_agg_"var"__chunk_len_5__attr_"stderr"',
       'value__approximate_entropy__m_2__r_0.1',
       'value__approximate_entropy__m_2__r_0.3',
       'value__approximate_entropy__m_2__r_0.5',
       'value__approximate_entropy__m_2__r_0.7',
       'value__approximate_entropy__m_2__r_0.9',
       'value__ar_coefficient__k_10__coeff_0',
       'value__ar_coefficient__k_10__coeff_1',
       'value__ar_coefficient__k_10__coeff_2',
       'value__ar_coefficient__k_10__coeff_3',
       'value__ar_coefficient__k_10__coeff_4',
       'value__augmented_dickey_fuller__attr_"pvalue"',
       'value__augmented_dickey_fuller__attr_"teststat"',
       'value__augmented_dickey_fuller__attr_"usedlag"',
       'value__autocorrelation__lag_1', 'value__autocorrelation__lag_2',
       'value__autocorrelation__lag_3', 'value__autocorrelation__lag_4',
       'value__autocorrelation__lag_5', 'value__autocorrelation__lag_6',
       'value__autocorrelation__lag_7', 'value__autocorrelation__lag_8',
       'value__autocorrelation__lag_9',
       'value__binned_entropy__max_bins_10', 'value__c3__lag_1',
       'value__c3__lag_2', 'value__c3__lag_3',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',
       'value__cid_ce__normalize_False', 'value__cid_ce__normalize_True',
       'value__count_above_mean', 'value__count_below_mean',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_2__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_2',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_1',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_2',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_3',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_5',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_6',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_7',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
       'value__energy_ratio_by_chunks__num_segments_10__segment_focus_9',
       'value__fft_aggregated__aggtype_"centroid"',
       'value__fft_aggregated__aggtype_"kurtosis"',
       'value__fft_aggregated__aggtype_"skew"',
       'value__fft_aggregated__aggtype_"variance"',
       'value__fft_coefficient__coeff_0__attr_"abs"',
       'value__fft_coefficient__coeff_0__attr_"real"',
       'value__fft_coefficient__coeff_10__attr_"abs"',
       'value__fft_coefficient__coeff_10__attr_"angle"',
       'value__fft_coefficient__coeff_10__attr_"imag"',
       'value__fft_coefficient__coeff_10__attr_"real"',
       'value__fft_coefficient__coeff_11__attr_"abs"',
       'value__fft_coefficient__coeff_11__attr_"angle"',
       'value__fft_coefficient__coeff_11__attr_"imag"',
       'value__fft_coefficient__coeff_11__attr_"real"',
       'value__fft_coefficient__coeff_12__attr_"abs"',
       'value__fft_coefficient__coeff_12__attr_"angle"',
       'value__fft_coefficient__coeff_12__attr_"imag"',
       'value__fft_coefficient__coeff_12__attr_"real"',
       'value__fft_coefficient__coeff_13__attr_"abs"',
       'value__fft_coefficient__coeff_13__attr_"angle"',
       'value__fft_coefficient__coeff_13__attr_"imag"',
       'value__fft_coefficient__coeff_13__attr_"real"',
       'value__fft_coefficient__coeff_14__attr_"abs"',
       'value__fft_coefficient__coeff_14__attr_"angle"',
       'value__fft_coefficient__coeff_14__attr_"imag"',
       'value__fft_coefficient__coeff_14__attr_"real"',
       'value__fft_coefficient__coeff_15__attr_"abs"',
       'value__fft_coefficient__coeff_15__attr_"angle"',
       'value__fft_coefficient__coeff_15__attr_"imag"',
       'value__fft_coefficient__coeff_15__attr_"real"',
       'value__fft_coefficient__coeff_16__attr_"abs"',
       'value__fft_coefficient__coeff_16__attr_"angle"',
       'value__fft_coefficient__coeff_16__attr_"imag"',
       'value__fft_coefficient__coeff_16__attr_"real"',
       'value__fft_coefficient__coeff_17__attr_"abs"',
       'value__fft_coefficient__coeff_17__attr_"angle"',
       'value__fft_coefficient__coeff_17__attr_"imag"',
       'value__fft_coefficient__coeff_17__attr_"real"',
       'value__fft_coefficient__coeff_18__attr_"abs"',
       'value__fft_coefficient__coeff_18__attr_"angle"',
       'value__fft_coefficient__coeff_18__attr_"imag"',
       'value__fft_coefficient__coeff_18__attr_"real"',
       'value__fft_coefficient__coeff_19__attr_"abs"',
       'value__fft_coefficient__coeff_19__attr_"real"',
       'value__fft_coefficient__coeff_1__attr_"abs"',
       'value__fft_coefficient__coeff_1__attr_"angle"',
       'value__fft_coefficient__coeff_1__attr_"imag"',
       'value__fft_coefficient__coeff_1__attr_"real"',
       'value__fft_coefficient__coeff_20__attr_"abs"',
       'value__fft_coefficient__coeff_20__attr_"angle"',
       'value__fft_coefficient__coeff_20__attr_"real"',
       'value__fft_coefficient__coeff_2__attr_"abs"',
       'value__fft_coefficient__coeff_2__attr_"angle"',
       'value__fft_coefficient__coeff_2__attr_"imag"',
       'value__fft_coefficient__coeff_2__attr_"real"',
       'value__fft_coefficient__coeff_3__attr_"abs"',
       'value__fft_coefficient__coeff_3__attr_"angle"',
       'value__fft_coefficient__coeff_3__attr_"imag"',
       'value__fft_coefficient__coeff_3__attr_"real"',
       'value__fft_coefficient__coeff_4__attr_"abs"',
       'value__fft_coefficient__coeff_4__attr_"angle"',
       'value__fft_coefficient__coeff_4__attr_"imag"',
       'value__fft_coefficient__coeff_4__attr_"real"',
       'value__fft_coefficient__coeff_5__attr_"abs"',
       'value__fft_coefficient__coeff_5__attr_"angle"',
       'value__fft_coefficient__coeff_5__attr_"imag"',
       'value__fft_coefficient__coeff_5__attr_"real"',
       'value__fft_coefficient__coeff_6__attr_"abs"',
       'value__fft_coefficient__coeff_6__attr_"angle"',
       'value__fft_coefficient__coeff_6__attr_"imag"',
       'value__fft_coefficient__coeff_6__attr_"real"',
       'value__fft_coefficient__coeff_7__attr_"abs"',
       'value__fft_coefficient__coeff_7__attr_"angle"',
       'value__fft_coefficient__coeff_7__attr_"imag"',
       'value__fft_coefficient__coeff_7__attr_"real"',
       'value__fft_coefficient__coeff_8__attr_"abs"',
       'value__fft_coefficient__coeff_8__attr_"angle"',
       'value__fft_coefficient__coeff_8__attr_"imag"',
       'value__fft_coefficient__coeff_8__attr_"real"',
       'value__fft_coefficient__coeff_9__attr_"abs"',
       'value__fft_coefficient__coeff_9__attr_"angle"',
       'value__fft_coefficient__coeff_9__attr_"imag"',
       'value__fft_coefficient__coeff_9__attr_"real"',
       'value__first_location_of_maximum',
       'value__first_location_of_minimum',
       'value__friedrich_coefficients__m_3__r_30__coeff_0',
       'value__friedrich_coefficients__m_3__r_30__coeff_1',
       'value__friedrich_coefficients__m_3__r_30__coeff_2',
       'value__friedrich_coefficients__m_3__r_30__coeff_3',
       'value__has_duplicate', 'value__has_duplicate_max',
       'value__has_duplicate_min', 'value__index_mass_quantile__q_0.1',
       'value__index_mass_quantile__q_0.2',
       'value__index_mass_quantile__q_0.3',
       'value__index_mass_quantile__q_0.4',
       'value__index_mass_quantile__q_0.6',
       'value__index_mass_quantile__q_0.7',
       'value__index_mass_quantile__q_0.8',
       'value__index_mass_quantile__q_0.9', 'value__kurtosis',
       'value__large_standard_deviation__r_0.05',
       'value__large_standard_deviation__r_0.1',
       'value__large_standard_deviation__r_0.15000000000000002',
       'value__large_standard_deviation__r_0.2',
       'value__large_standard_deviation__r_0.25',
       'value__large_standard_deviation__r_0.30000000000000004',
       'value__large_standard_deviation__r_0.35000000000000003',
       'value__large_standard_deviation__r_0.4',
       'value__large_standard_deviation__r_0.45',
       'value__last_location_of_maximum',
       'value__last_location_of_minimum', 'value__length',
       'value__linear_trend__attr_"intercept"',
       'value__linear_trend__attr_"pvalue"',
       'value__linear_trend__attr_"rvalue"',
       'value__linear_trend__attr_"slope"',
       'value__linear_trend__attr_"stderr"',
       'value__longest_strike_above_mean',
       'value__longest_strike_below_mean',
       'value__max_langevin_fixed_point__m_3__r_30', 'value__maximum',
       'value__mean', 'value__mean_abs_change', 'value__mean_change',
       'value__median', 'value__minimum', 'value__number_crossing_m__m_1',
       'value__number_cwt_peaks__n_1', 'value__number_cwt_peaks__n_5',
       'value__number_peaks__n_1', 'value__number_peaks__n_10',
       'value__number_peaks__n_3', 'value__number_peaks__n_5',
       'value__partial_autocorrelation__lag_1',
       'value__partial_autocorrelation__lag_2',
       'value__partial_autocorrelation__lag_3',
       'value__partial_autocorrelation__lag_4',
       'value__partial_autocorrelation__lag_5',
       'value__partial_autocorrelation__lag_6',
       'value__partial_autocorrelation__lag_7',
       'value__partial_autocorrelation__lag_8',
       'value__partial_autocorrelation__lag_9',
       'value__percentage_of_reoccurring_datapoints_to_all_datapoints',
       'value__percentage_of_reoccurring_values_to_all_values',
       'value__quantile__q_0.1', 'value__quantile__q_0.2',
       'value__quantile__q_0.3', 'value__quantile__q_0.4',
       'value__quantile__q_0.6', 'value__quantile__q_0.7',
       'value__quantile__q_0.8', 'value__quantile__q_0.9',
       'value__range_count__max_1__min_-1',
       'value__ratio_beyond_r_sigma__r_0.5',
       'value__ratio_beyond_r_sigma__r_1',
       'value__ratio_beyond_r_sigma__r_1.5',
       'value__ratio_beyond_r_sigma__r_2',
       'value__ratio_beyond_r_sigma__r_2.5',
       'value__ratio_beyond_r_sigma__r_3',
       'value__ratio_beyond_r_sigma__r_5',
       'value__ratio_beyond_r_sigma__r_6',
       'value__ratio_value_number_to_time_series_length',
       'value__sample_entropy', 'value__skewness',
       'value__spkt_welch_density__coeff_2',
       'value__spkt_welch_density__coeff_5',
       'value__spkt_welch_density__coeff_8', 'value__standard_deviation',
       'value__sum_of_reoccurring_data_points',
       'value__sum_of_reoccurring_values', 'value__sum_values',
       'value__symmetry_looking__r_0.05',
       'value__symmetry_looking__r_0.1',
       'value__symmetry_looking__r_0.15000000000000002',
       'value__symmetry_looking__r_0.2',
       'value__symmetry_looking__r_0.25',
       'value__symmetry_looking__r_0.30000000000000004',
       'value__symmetry_looking__r_0.35000000000000003',
       'value__symmetry_looking__r_0.4',
       'value__symmetry_looking__r_0.45',
       'value__symmetry_looking__r_0.5',
       'value__symmetry_looking__r_0.55',
       'value__symmetry_looking__r_0.6000000000000001',
       'value__symmetry_looking__r_0.65',
       'value__symmetry_looking__r_0.7000000000000001',
       'value__symmetry_looking__r_0.75',
       'value__symmetry_looking__r_0.8',
       'value__symmetry_looking__r_0.8500000000000001',
       'value__symmetry_looking__r_0.9',
       'value__symmetry_looking__r_0.9500000000000001',
       'value__time_reversal_asymmetry_statistic__lag_1',
       'value__time_reversal_asymmetry_statistic__lag_2',
       'value__time_reversal_asymmetry_statistic__lag_3',
       'value__variance',
       'value__variance_larger_than_standard_deviation']
TSFRESH_RELEVANT

## Other stuff

In [ ]:
X_tsfresh, augmented_df = add_tsfresh_features(df_train_x)

In [ ]:
# relevant features = 326
# all_features = 794
augmented_df[augmented_df['quoteDate']=='2018-06-04']

In [ ]:
# X_tsfresh.index.name
X_tsfresh[X_tsfresh.index=='2018-06-04']

In [ ]:
df_train_x.describe()

In [ ]:
df_time_series = pd.DataFrame()
df_time_series['symbol'] = df_train_x['symbol']
df_time_series['quoteDate'] = df_train_x['quoteDate']
df_time_series['adjustedPrice'] = df_train_x['adjustedPrice']
df_time_series.dropna(axis=0, how='any', inplace=True)
# df_time_series.set_index('quoteDate', inplace=True)
df_time_series

In [ ]:
df_shift, y = make_forecasting_frame(df_time_series["adjustedPrice"], kind="return", 
                                     max_timeshift=40, rolling_direction=1)

In [ ]:
df_shift

In [ ]:
y

In [ ]:
df_time_series['quoteDate']

In [ ]:
ts_X = roll_time_series(df_time_series, column_id="symbol", column_sort="quoteDate", 
                        column_kind="adjustedPrice", max_timeshift=40, 
                        rolling_direction=1)

In [ ]:
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh import extract_features, extract_relevant_features, select_features

extraction_settings = ComprehensiveFCParameters()
ts_X = extract_features(df_time_series, 
                        column_id='symbol', column_sort='quoteDate',
                        default_fc_parameters=extraction_settings)

In [ ]:
ts_X

In [ ]:
from tsfresh.transformers import RelevantFeatureAugmenter

augmenter = RelevantFeatureAugmenter()
augmenter.set_timeseries_container(train_time_series)
augmenter.fit(X_train, y_train)


In [ ]:
X = extract_relevant_features(df_shift, y, column_id="id", column_sort="time", column_value="value", 
                              # impute_function=impute, 
                              ml_task='regression', show_warnings=True)

In [ ]:
X

In [ ]:
df_time_series[df_time_series.isnull().any(axis=1)]

In [ ]:
y = pd.DataFrame(data={
    'adjustedPrice': df_time_series['adjustedPrice'].values
    },
                 index=df_time_series[['symbol', 'quoteDate']])
y['adjustedPrice']

In [ ]:
X = extract_relevant_features(df_time_series, y['adjustedPrice'], 
                              column_id='symbol', column_sort='quoteDate', show_warnings=True)

In [ ]:
unseen_df = pd.read_pickle('../data/ml-20180922-unlabelled.pkl.gz', compression='gzip')

In [ ]:
mask_6_weeks = (unseen_df['quoteDate'] < '2018-07-21')
mask_7_weeks = (unseen_df['quoteDate'] >= '2018-07-21') & (unseen_df['quoteDate'] < '2018-07-28')
mask_6_weeks

In [ ]:
unseen_df.loc[mask_6_weeks]['future_eight_week_return']